<font size="6">Линейный классификатор </font>

# Ограничения алгоритма k-nearest neighbors (kNN)



Продолжим с того места, где мы остановились в предыдущей лекции - классификация методом ближайших соседей (kNN). 

**Метрики расстояния**

В практических заданиях EX01 мы использовали kNN для классификации изображений CIFAR10 (набор фотографий разделенных на 10 классов). Мы выяснили, что точность метода классификации с помощью kNN с использованием расстояний L1 (*Manhatten distance* - сумма абсолютных разностей между пикселями) оставляет желать лучшего, и попробовали применить L2 (*Euclidian distance*) в надежде эту точность повысить. С метриками L1 и L2 мы будем сталкиваться часто: и в качестве loss функции, и в качестве регуляризации, поэтому познакомиться с ними полезно. 

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/out/l1_manhattan_and_l2_euclidian_distance.png" width="600">

## kNN для классификации


На практике, метод ближайших соседей для классификации используется крайне редко. Давайте разберемся почему.
Проблема заключается в следующем: предположим, что точность классификации нас устраивает. Теперь давайте применим kNN на больших данных (e.g. миллион картинок). Для определения класса каждой из картинок, нам нужно сравнить ее со всеми другими картинками в базе данных, а такие расчеты, даже в существенно оптимизированном виде, занимают много времени. Мы же хотим, чтобы обученная модель работала быстро.

Тем не менее, метод ближайших соседей используется в других задачах, где без него обойтись сложно. Например, в задаче распознавания лиц. Представим, что у нас у нас есть большая база данных с фотографиями лиц (например, по 5 разных фотографий всех сотрудников, которые работают в офисном здании, как на примере выше) и есть камера, установленная на входе в это здание. Мы хотим узнать, кто и во сколько пришел на работу. Для того чтобы понять кто прошел перед камерой, нам нужно зафиксировать лицо этого человека и сравнить его со всеми фотографиями лиц в базе. В такой формулировке мы не пытаемся определить конкретный класс фотографии, а всего лишь определяем “похож-не похож”. Мы смотрим на k ближайших соседей и, например, если из k соседей, 5 - это фотографии Джеки Чана, то, скорее всего, под камерой прошел именно он. В таких случаях kNN метод вполне полезен. Похожим образом работает и поиск дубликатов в базах данных.

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/out/knn_for_classification.png" width="450">

Примеры эффективной реализации метода на основе kNN:
* [Facebook AI Research Similarity Search](https://github.com/facebookresearch/faiss) – разработка команды Facebook AI Research для быстрого поиска ближайших соседей и кластеризации в векторном пространстве. Высокая скорость поиска позволяет работать с очень большими данными – до нескольких миллиардов векторов.
* Алгоритм поиска ближайших соседей [Hierarchical Navigable Small World](https://arxiv.org/abs/1603.09320). 

## Практические аспекты работы с классификаторами

### Нормализация данных

Загрузим датасет с образцами здоровой и раковой ткани. Датасет состоит из 569 примеров, где каждой строчке из 30 признаков, соответствует класс `1` злокачественной (*malignant*) или `0` доброкачественной (*benign*) ткани. Задача состоит в том, чтобы по 30 признакам обучить модель определять тип ткани (злокачественная или доброкачественная).

Можно иметь сколь угодно хороший алгоритм для классификации - но до тех пор, пока данные на входе - мусор, на выходе из нашего чудесного классификатора мы тоже будем получать мусор (*garbage in - garbage out*). Давайте разберемся, что конкретно надо сделать, чтобы kNN реально заработал.


In [ ]:
import sklearn.datasets

cancer = sklearn.datasets.load_breast_cancer() # load data
x = cancer.data # features
y = cancer.target # labels(classes)
print(f'x shape: {x.shape}, y shape: {y.shape}') 
print(f'x[0]: \n {x[0]}') 
print(f'y[0]: \n {y[0]}') 

Посмотрим сколько данных в классе `0` и сколько данных в классе `1`

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 5)) # set fig size 
plt.bar(1, y[y == 1].shape, label=cancer.target_names[0]) # 1 label 
plt.bar(0, y[y == 0].shape, label=cancer.target_names[1]) # 0 label
plt.title('Class balance')
plt.ylabel('Num examples')
plt.xticks(ticks=[1, 0], labels=['1', '0']) 
plt.legend(loc='upper left') 
plt.show() 

Теперь давайте посмотрим на сами данные. У нас есть 569 строк в каждой, из которой, по 30 колонок. Такие колонки называют признаками или *features*. Попробуем математически описать все эти признаки (mean, std, min и тд)

In [ ]:
import pandas as pd
pd.DataFrame(x).describe()

То же самое, но в виде графика. Видно, что у фич совершенно разные значения.

In [ ]:
import seaborn as sns

ax = sns.boxenplot(data=pd.DataFrame(x), orient="h", palette="Set2")
ax.set(xscale='log', xlim=(1e-4, 1e4), xlabel='Values', ylabel='Features')
plt.show()

Чтобы адекватно сравнить данные между собой нам следует использовать нормализацию.

 **Нормализация, выбор Scaler**

Нормализацией называется процедура приведения входных данных к единому масштабу (диапазону) значений. Фактически, это означает построение взаимно однозначного соответствия между некоторыми размерными величинами (которые измеряются в метрах, килограммах, годах и т. п.) и их безразмерными аналогами, принимающими значение в строго определенном числовом диапазоне (скажем, на отрезке $[0,1]$). Преобразование данных к единому числовому диапазону (иногда говорят *домену*) позволяет считать их равноправными признаками и единообразно передавать их на вход модели. В некоторых источниках данная процедура явно называется *масштабирование*.

$$\text{scaling map} \; : \text{some arbitrary feature domain} \rightarrow \text{definite domain} $$

Иногда под нормализацией данных понимают процедуру *стандартизации*, то есть приведение множеств значений всех признаков к стандартному нормальному распределению -- распределению, с нулевым среднем значением и единичной дисперсией.

$$\text{standartization map} : f_i \rightarrow (f_i - \text{mean} (\{f_i\})) \cdot \frac{1}{\text{std} (\{f_i\})}$$


Рассмотрим небольшой пример. Пусть у нас есть данные о некоторой группе людей, содержащие два признака: *возраст* (в годах) и *размер дохода* (в рублях). Возраст может измениться в диапазоне от 18 до 70 ( интервал 70-18 = 52). А доход от 30 000 р до 500 000 р (интервал 500 000 - 30 000 = 470 000). В таком варианте разница в возрасте имеет меньшее влияние, чем разница в доходе. Получается, что доход становится более важным признаком, изменения в котором влияют больше при сравнении схожести двух людей.

Должно быть так, чтобы максимальные изменения любого признака в «основной массе объектов» были одинаковы. Тогда потенциально все признаки будут равноценны.

Осталось определиться с выбором инструмента, часто используют следующие варианты: `MinMaxScaler`, `StandardScaler`, `RobustScaler`

Сравним `MinMaxScaler`, `StandardScaler`, `RobustScaler` для признака `data[:,0]`. **Обратите внимание на ось X**

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import numpy as np

np.random.seed(42)  # setting the initialization parameter for random values

# generate random values from 1 to 255, shape (30,1)
test = x[:, 0].reshape(-1, 1)

plt.figure(1, figsize=(30, 5))  
plt.subplot(141)  # set location
plt.scatter(test, range(len(test)), c=y)  
plt.ylabel("Num examples", fontsize=15)  
plt.xticks(fontsize=15)  
plt.yticks(fontsize=15)  
plt.title("Non scaled data", fontsize=18)  

# scale data with MinMaxScaler
test_scaled = MinMaxScaler().fit_transform(test)  
plt.subplot(142)
plt.scatter(test_scaled, range(len(test)), c=y)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("MinMaxScaler", fontsize=18)

# scale data  with StandardScaler
test_scaled = StandardScaler().fit_transform(test)  
plt.subplot(143)
plt.scatter(test_scaled, range(len(test)), c=y)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("StandardScaler", fontsize=18)

# scale data  with RobustScaler
test_scaled = RobustScaler().fit_transform(test)  
plt.subplot(144)
plt.scatter(test_scaled, range(len(test)), c=y)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("RobustScaler", fontsize=18)
plt.show()

Идея **`MinMaxScaler`** заключается в том, что он преобразует данные в диапазоне от 0 до 1. Может быть полезно, если нужно выполнить преобразование, в котором отрицательные значения не допускаются (e.g., масштабирование RGB пикселей)


$$z=\frac{X_i-X_{min}}{X_{max}-X_{min}}$$

$X_{min}$ и $X_{max}$ задаются как минимальное и максимальное допустимое значение, по умолчанию:  $X_{min}=0$  и $X_{max}=1$

Идея **`StandardScaler`** заключается в том, что он преобразует данные таким образом, что распределение будет иметь среднее значение 0 и стандартное отклонение 1. Большинство значений будет в  диапазоне от -1 до 1. Это стандартная трансформация, и она применима во многих ситуациях.

$$z=\frac{X-u}{s}$$

$u$ — среднее значение (или 0 при `with_mean=False`) и $s$ — стандартное отклонение (или 0 при `with_std=False`)

И StandardScaler и MinMaxScaler очень чувствительны к наличию выбросов. **`RobustScaler`** использует медиану и основан на *процентилях*. k-й процентиль – это величина, равная или не превосходящая k процентов чисел во всем имеющемся распределении. Например, 50-й процентиль (медиана) распределения таково, что 50% чисел из распределения не меньше данного числа. Соответственно, RobustScaler не зависит от небольшого числа очень больших предельных выбросов (outliers). Следовательно, результирующий диапазон преобразованных значений признаков больше, чем для предыдущих скэйлеров и, что более важно, примерно одинаков.

$$z=\frac{X-X_{median}}{IQR}$$

$X_{median}$ — значение медианы, $IQR$ — межквартильный диапазон равный разнице между 75-ым и 25-ым процентилями

Для нашей задачи по определению раковых опухолей обработаем наши 30 признаков с помощью StandardScaler.

In [ ]:
x_norm = StandardScaler().fit_transform(x)  # scaled data

Видим, что они стали намного более сравнимы между собой.

In [ ]:
pd.DataFrame(x_norm).describe()

In [ ]:
ax = sns.boxenplot(data=pd.DataFrame(x_norm), 
                   orient="h", 
                   palette="Set2")
ax.set(xlabel='Values', ylabel='Features')
plt.show()

### Переобучение

Теперь обучим kNN для общей выборки данных, при разном значении количества соседей.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

n_nei_rng = np.arange(1, 31)  # array of the number of neighbors

quality = np.zeros(
    n_nei_rng.shape[0]
)  

for ind in range(n_nei_rng.shape[0]):  # for all elements
    # create knn for all num neighbors 
    knn = KNeighborsClassifier(
        n_neighbors=n_nei_rng[ind]
    )  
    knn.fit(x_norm, y)  
    q = accuracy_score(y_pred=knn.predict(x_norm), y_true=y)  # accuracy
    quality[ind] = q  # fill quality

plt.figure(figsize=(8, 5))  
plt.title("KNN on train", size=20)  
plt.xlabel("Neighbors", size=15)  
plt.ylabel("Accuracy", size=15)  
plt.plot(n_nei_rng, quality)  
plt.xticks(n_nei_rng) 
plt.show()  

Видим, что качество на 1 соседе - самое лучшее. Но это и понятно - ближайшим соседом элемента из обучающей выборки будет сам объект. Мы просто **запомнили** все объекты.

Если теперь мы попробуем взять какой-то новый образец опухоли и классифицировать его - у нас скорее всего ничего не получится. В таких случаях мы говорим, что наша модель не умеет обобщать (*generalization*).

Для того, чтобы знать заранее обобщает ли наша модель или нет, мы можем разбить все имеющиеся у нас данныe на 2 части. Но одной части мы будем обучать классификатор (*train set*), а на другой тестировать насколько хорошо он работает (*test set*).

In [ ]:
from sklearn.model_selection import train_test_split

# split data to train/test
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)

scaler = StandardScaler()  
scaler.fit(x_train)  
x_train_norm = scaler.transform(x_train)  # scaling data
x_test_norm = scaler.transform(x_test)  # scaling data

n_nei_rng = np.arange(1, 31)  
train_quality = np.zeros(n_nei_rng.shape[0])  # quality on train data
test_quality = np.zeros(n_nei_rng.shape[0])  # quality on test data

for ind in range(n_nei_rng.shape[0]):  
    knn = KNeighborsClassifier(n_neighbors=n_nei_rng[ind])  
    knn.fit(x_train_norm, y_train)  
    
    # accuracy on train data
    trq = accuracy_score(y_pred=knn.predict(x_train_norm), y_true=y_train)  
    train_quality[ind] = trq  

    # accuracy on test data
    teq = accuracy_score(y_pred=knn.predict(x_test_norm), y_true=y_test)  
    test_quality[ind] = teq  

# accuracy plot  on train and test data
plt.figure(figsize=(8, 5))
plt.title("KNN on train vs test", size=20)
plt.plot(n_nei_rng, train_quality, label="train")
plt.plot(n_nei_rng, test_quality, label="test")
plt.legend()
plt.xticks(n_nei_rng)
plt.xlabel("Neighbors", size=15)
plt.ylabel("Accuracy", size=15)
plt.show()

Вот, теперь мы видим, что 1 сосед был "ложной тревогой". Такие случаи мы называем *переобучением*. Чтобы действительно предсказывать что-то полезное, нам надо выбирать число соседей, начиная минимум с 3

### Кросс-валидация

Каждая модель имеет как ряд **параметров**, которые она меняет в процессе обучения (например, веса модели), так и ряд **гиперпараметров**, которые влияют на то, каким способом модель меняет параметры в процессе обучения. 

В случае kNN параметры, строго говоря, отсутствует - модель просто запоминает объекты обучающей выборки. Особо упорные могут считать их параметрами. 

А вот гиперпараметры есть, даже несколько групп. Какие? 

1. Число соседей 
2. Функция, которой считаем расстояние между объектами (L2=eucledian, L1=manhattan)
3. Веса, с которыми складываем метки ближайших соседей
4. Признаки! (но об этом с вами поговорим позже)
5. Сама модель - мы могли выбрать не kNN, а нагуглить что-нибудь другое

Посмотрим еще раз на график, который нарисовали на прошлом шаге. Какое число соседей считать оптимальным? Метрика явно скачет? 

In [ ]:
plt.figure(figsize=(8, 5))
plt.title("KNN on train vs test", size=20)
plt.plot(n_nei_rng, train_quality, label="train")
plt.plot(n_nei_rng, test_quality, label="test")
plt.xlabel("Neighbors", size=15)
plt.ylabel("Accuracy", size=15)
plt.legend()
plt.xticks(n_nei_rng)
plt.show()


Не понятно, насколько результат зависит от того, как нам повезло или не повезло с разбиением данных на обучение и тест. Может оказаться так, что для конкретного разбиения хорошо выбрать k=5, а для другого - k=7. 

Кроме того, опять же - фактически, мы сами выступаем в роли модели, которая учит гиперпараметры (а не параметры) под видимую ей выборку. 

Представим себе, что у нас есть 10000 моделей, полученных подкручиванием разных гиперпараметров (в том числе, выбором просто разного типа модели). Представим, что все эти модели не работают. Вообще. Представим так же, что каждая модель угадывает класс в задаче разделения на два класса с вероятностью 0.5 (и будем считать, что классы у нас сбалансированны - то есть 50% одного класса и 50% другого). 
Опять же, понятно, что классификация такими моделями ничем не лучше подбрасывания монетки. 



In [ ]:
def guess_model(y_real):
    # array with values True/False, for random choice by mask 
    guessed = np.random.choice(
        [True,False],  
        size=y_real.shape[0],
        replace=True,
    )
    y_pred = np.zeros_like(y_real)  # zeros array, shape y_real
    
    # with mask 'guessed' assign values 
    y_pred[guessed] = y_real[guessed]  
    y_pred[~guessed] = 1 - y_real[~guessed]  
    return y_pred

In [ ]:
models_num = 10000  # num of experiments
best_quality = 0.5  # quality threshold for accuracy

# array with random values in range 0, 1
y_real = np.random.choice(
    [0, 1], size=250, replace=True
)  

for i in range(models_num):  # for all expirements
    y_pred = guess_model(y_real)  # predicted values
    q = accuracy_score(y_pred=y_pred, y_true=y_real)  # accuracy
    if q > best_quality:  
        best_quality = q   
print(f"Best result {best_quality}")

То есть мы перебором всех возможных моделей вполне можем получить для абсолютно бесполезной модели приемлемое качество


Получается, что если подбирать гиперпараметры модели на *train set*, то:
1. Можно переобучитьcя, просто на более "высоком" уровне. Особенно если гиперпараметров у модели много и все они разнообразны
2. Нельзя быть уверенным, что выбор параметров не зависит от разбиения на обучение и тест 

Поэтому мы:

1. Подбираем гиперпараметры моделей на отдельном датасете, называемым валидационным. Получаем мы его разбиением обучающего датасета на собственно обучающий и валидационный 

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/out/split_dataset_for_train_val_test.png" width="700">

2. Чаще всего делаем несколько таких разбиений по какой-то схеме, чтобы получить уверенность оценок качества для моделей с разными гиперпараметрами - **кросс-валидация**

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/out/cross_validation_on_train_data.png" width="500">

Часто применяется следующий подход, называемый [K-Fold кросс-валидацией](https://scikit-learn.org/stable/modules/cross_validation.html):

Берется тренировочная часть датасета, разбивается на части - блоки. Дальше мы будем использовать для проверки первую часть (Fold 1), а на остальных учиться. И так последовательно для всех частей. В результате у нас будут информация о точности для разных фрагментов данных и уже на основании этого можно понять, насколько значение этого параметра, который мы проверяем, зависит или не зависит от данных. То есть если у нас от разбиения точность при одном и том же К меняться не будет, значит мы подобрали правильное К. Если она будет сильно меняться в зависимости от того, на каком куске данных мы проводим тестирование, значит, надо попробовать другое К и если ни при каком не получилось - то это такие данные.  


Используем инструмент `GridSearchCV` для подбора оптимальных параметров модели.

`GridSearchCV` автоматически перебирает все возможные комбинации из переданного ему  множества возможного параметров, для каждой такой возможной комбинации параметров проводит описанную выше процедуру K-Fold кросс-валидации и запоминает модель с лучшей комбинацией параметров.

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold
from IPython.display import clear_output
"""
Parameters for GridSearchCV:
estimator — model
cv — num of fold to cross-validation splitting 
param_grid — parameters names
scoring — metrics 
n_jobs - number of jobs to run in parallel, -1 means using all processors.
"""
model = GridSearchCV(
    estimator=KNeighborsClassifier(),
    cv=KFold(3, shuffle=True, random_state=42),
    param_grid={
        "n_neighbors": np.arange(1, 31),
        "metric": ["euclidean", "manhattan"],
        "weights": ["uniform", "distance"],
    },
    scoring="accuracy",
    n_jobs =- 1,
)
model.fit(x_train_norm, y_train)
clear_output()

Выведем лучшие гиперпараметры для модели, которые подобрали:

In [ ]:
print("Metric:", model.best_params_["metric"])
print("Num neighbors:", model.best_params_["n_neighbors"])
print("Weigths:", model.best_params_["weights"])


Объект GridSearchCV можно использовать как обычную модель

In [ ]:
from sklearn.metrics import balanced_accuracy_score

y_pred = model.predict(x_test_norm)
print(f"Percent correct predictions {np.round(accuracy_score(y_pred=y_pred, y_true=y_test) * 100, 2)} %")
print(f"Percent correct predictions(balanced classes) {np.round(balanced_accuracy_score(y_pred=y_pred, y_true=y_test) * 100, 2)} %")

Мы можем извлечь дополнительные данные о кроссвалидации и по ключу обратиться к результатам всех моделей

In [ ]:
list(model.cv_results_.keys())

Выведем для примера mean_test_score:

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(model.cv_results_["mean_test_score"])
plt.title("mean_test_score", size=20)
plt.xlabel("Num of experiment", size=15)
plt.ylabel("Accuracy", size=15)
plt.show()

Построим, например, при фиксированных остальных параметрах (равных лучшим параметрам), качество модели на валидации в зависимости от числа соседей

In [ ]:
selected_means = []
selected_std = []
n_nei = []
for ind, params in enumerate(model.cv_results_["params"]):
    if (
        params["metric"] == model.best_params_["metric"]
        and params["weights"] == model.best_params_["weights"]
    ):
        n_nei.append(params["n_neighbors"])
        selected_means.append(model.cv_results_["mean_test_score"][ind])
        selected_std.append(model.cv_results_["std_test_score"][ind])

Построим error bar, для сравнения разброса ошибки при разном количестве соседей Neighbors

In [ ]:
plt.figure(figsize=(8, 5))
plt.title(f"KNN CV, {params['metric']}, {params['weights']}", size=20)
plt.errorbar(n_nei, selected_means, yerr=selected_std, linestyle="None", fmt="-o")
plt.xticks(n_nei)
plt.ylabel("Mean_test_score", size=15)
plt.xlabel("Neighbors", size=15)

plt.show()

Видим, что на самом деле большой разницы в числе соседей и нет. 

**Можно ли делать только кросс-валидацию (без теста)?**


Нет, нельзя. Кросс-валидация не до конца спасает от подгона параметров модели под выборку, на которой она проводится. Оценка конечного качества модели должно производиться на отложенной тестовой выборке. Если у вас очень мало данных, можно рассмотреть [вложенную кросс-валидацию](https://machinelearningmastery.com/nested-cross-validation-for-machine-learning-with-python/), Речь об этом пойдет позже, в последующих лекциях. Но даже в этом случае придется анализировать поведение модели, чтобы показать, что она учит что-то разумное. Кстати, вложенную кросс-валидацию можно использовать, чтобы просто получить более устойчивую оценку поведения модели на тесте.

# Линейный классификатор

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/out/use_knn_to_comare_imgs_to_find_similar.png" width="800">

Давайте подумаем, как избавиться от проблемы со скоростью K-Nearest Neighbors. Реализуя метод ближайшего соседа, мы сравнивали одно изображение со всеми, при этом мы предполагали, что изображения из одного класса будут чем-то похожи друг на друга, и поэтому разница будет меньше. Метод как-то работал. 

Чтобы ускорить этот процесс, мы можем взять весь блок изображений (справа), который относится, например, к машинам и усредним. Таким образом, получим некоторый шаблон для класса “автомобиль”. Скорее всего, работать такой подход будет не слишком здорово, но зато вместо тысяч изображений (в данном случае 50000) появится одно. Возможно, это поможет сильно сэкономить время.

## Переход к сравнению с шаблоном

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/out/one_template_for_each_class_for_classification.png" width="450">

Идея в следующем: вместо того, чтобы сравнивать каждое изображение со всеми остальными по очереди, будем сравнивать изображения с шаблоном для каждого класса. Их будет всего 10 для данного датасета. Этот подход позволит радикально увеличить скорость. 

Проверим, что получится из этой идеи. 


Cкачиваем CIFAR10

In [ ]:
from torchvision.datasets import CIFAR10
import numpy as np

dataset_train = CIFAR10("content", train=True, download=True)
dataset_test = CIFAR10("content", train=False, download=True)

x_train = dataset_train.data
y_train = np.array(dataset_train.targets)

x_test = dataset_test.data
y_test = np.array(dataset_test.targets)

print(f"x_train shape: {x_train.shape}, y_train shape: {y_train.shape}")
print(f"x_train shape : {x_test.shape}, y_test shape: {y_test.shape}")

Cоздадим шаблоны посчитав среднее значение пикселя по всем изображениям одного класса.


In [ ]:
templates = []  

for i in range(len(dataset_train.classes)):
    imgs = x_train[y_train == i] # select images by class mask
    mn = np.mean(imgs, 0)  
    templates.append(mn.astype(int)) # convert to int for display

Визуализируем полученные шаблоны:

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (20.0, 2.0))

def show_templates(templates, labels):
    for i, template in enumerate(templates):
        plt.subplot(1, len(labels), i + 1)
        plt.title(labels[i])
        plt.axis("off")
        plt.imshow(template)

show_templates(templates, dataset_train.classes)  

Теперь у нас есть 10 шаблонов, с которыми мы будем сравнивать изображение и на основании этого делать предсказание. На этих шаблонах можно увидеть очертания объектов.

Напишем классификатор, который будет сравнивать изображение с шаблоном, который генерируется во время обучения.

In [ ]:
class TemplateBasedClassifier():
    def __init__(self, labels):
        self.templates = []          
        self.labels = labels

    def fit(self, x_train, y_train):
        self.templates = []
        for label_num in range(len(self.labels)):  
          imgs = x_train[y_train == label_num] # select images by class mask
          mn = np.mean(imgs, 0 )  
          self.templates.append(mn)  

    def forward(self, x):
        # compute distance score
        distances = np.sum(np.abs(self.templates - x), axis=(1 ,2, 3))  
        return np.argmin(distances)  # return minimum score index

Рассмотрим сравнение на конкретном примере с кошкой. Построим изображения картинки, шаблона, а затем посчитаем расстояние L1 между ними. Чем желтее цвет - тем больше изображение похоже на шаблон

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig,ax = plt.subplots(ncols=3, figsize=(10, 5))

cat_class_num = 3
imgs = x_train[y_train == cat_class_num] # images class 3(cat)

img = imgs[1] # some image with cat
template = templates[cat_class_num] 
residual = np.mean(np.abs(img-template), axis = -1) 

# Code for display
ax[0].imshow(img)
ax[1].imshow(template)
r_plot = ax[2].imshow(residual, cmap='inferno_r')

divider = make_axes_locatable(ax[2])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(r_plot, cax=cax, orientation='vertical', label='L1')

ax[0].set_title('Image')
ax[1].set_title('Template')
ax[2].set_title('D = Image - Template')

for a in ax:
    a.axis('off')

In [ ]:
model = TemplateBasedClassifier(dataset_train.classes)  
model.fit(x_train, y_train)  

Сделаем предсказание и замерим время

In [ ]:
import time
from sklearn.metrics import accuracy_score

def validate(model, x_test, y_test):
    y_pred = []  # num of correct predictions
    for i, img in enumerate(x_test):  # for all images
        index = model.forward(img)  # predict class
        y_pred.append(index)
    return accuracy_score(y_test, y_pred)

start = time.perf_counter()  
accuracy= validate(model, x_test, y_test)  
tm = time.perf_counter() - start 
print(
    "\nAccuracy {:.2f} Train {:d} /test {:d} in {:.1f} sec. speed {:.2f} samples per second.".format(
        accuracy, len(x_train), len(x_test), tm, len(x_test) / tm
    )
)

Если бы мы обучали KNN на таком количестве данных, это заняло бы у нас четверть часа.



##  Переход к весам
----
Умножение вместо вычитания - для чего?


<img src ="https://edunet.kea.su/repo/EduNet-content/L02/out/compute_scalar_product_with_compare_with_template.png" width="450">

В сравнении с шаблонами, мы вычитали нашу картинку из шаблона и таким образом смотрели, насколько они похожи (L1). Теперь давайте сделаем следующий, пока ничем не обоснованный, ход: оставим всё, как было, но заменим вычитание умножением. Логика в том, что не все пиксели одинаково важны. Вероятно, если на изображении совпадут какие-то пиксели, которые отвечают, например, за глаза кошки, это будет намного важнее, чем фон, который может быть точно таким же у собаки. Здесь будут какие-то важные особенности, которым можно придать больший вес.

Если мы распишем в виде формул наложение шаблона на картинку таким образом с умножением, то получается, что мы скалярно перемножаем два вектора. Подробнее про [скалярное произведение](https://ru.wikipedia.org/wiki/Скалярное_произведение) векторов. Для того, чтобы получить вектор из изображения размерностью 32х32х3, достаточно "выпрямить" его, получим вектор размерностью 1х3072.


Опять рассмотрим сравнение на конкретном примере с кошкой. Построим изображения картинки, шаблона, а затем посчитаем расстояние между ними, используя наш новый метод перемножения. Чем желтее цвет - тем больше изображение похоже на шаблон

In [ ]:
fig,ax = plt.subplots(ncols=3, figsize=(10, 5))

residual = np.mean(np.abs(img * template), -1)

ax[0].imshow(img)
ax[1].imshow(template)
r_plot = ax[2].imshow(residual, cmap='inferno_r')

divider = make_axes_locatable(ax[2])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(r_plot, cax=cax, orientation='vertical', label='Distance')

ax[0].set_title('Image')
ax[1].set_title('Template')
ax[2].set_title('D = Image * Template')

for a in ax:
    a.axis('off')

###  Математическая запись

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/out/img_to_vector_to_compute_scalar_product.png" width="700">



Обозначим входное изображение как $x$, а шаблон для первого из классов как $w_0$.

Элементы пронумеруем подряд 1,2,3 … $n$. То есть развернем матрицу пикселей изображения в вектор. 

Тогда результат сравнения изображения с этим шаблоном будет вычисляться по формуле: $x[0]*w0[0] + x[1]*w0[1] + … x[n-1]*w0[n-1]$ 




<img src ="https://edunet.kea.su/repo/EduNet-content/L02/out/scalar_product_ways_to_use.png" width="800">


Эта простая модель лежит в основе практически всех сложных, которые мы будем рассматривать дальше. Внутри мы будем также пользоваться скалярным произведением.

В дальнейшем мы будем проходить сверточные сети, они работают очень похоже:
мы тоже накладываем шаблон на некоторую матрицу и перемножаем элементы, затем складываем. Единственное отличие – обычно ядро свертки меньше, чем размер самого изображения. 



## Support Vector Machine (метод опорных векторов)

[Отличное видео про SVM от Stat Quest которое все объясняет](https://www.youtube.com/watch?v=efR1C6CvhmE&ab_channel=StatQuestwithJoshStarmer)

### 1D классификация

Рассмотрим одномерный пример. У нас есть данные по массе мышей. Часть из них определена как нормальные, а часть как мыши с ожирением. Чтобы их отделить друг от друга, нам достаточно одного критерия. Мы можем посмотреть на график, и визуально определить предельную массу, после которой мышки будут жирненькими.

In [ ]:
import seaborn as sns

def generate_data(total_len=40):
    x = np.hstack([np.random.uniform(14, 21, total_len // 2), 
                   np.random.uniform(24, 33, total_len // 2)])
    y = np.hstack([np.zeros(total_len // 2), 
                   np.ones(total_len // 2)])
    return x, y

def plot_data(x, y, total_len=40, s=50, threshold=None, margin=None):
    ax = sns.scatterplot(x=x, y=np.zeros(len(x)), hue=y, s=s)
    if threshold:
        ax.axvline(threshold, color='red', ls='dashed')
    if margin:
        for line in margin:
            ax.axvline(line, color='pink', ls='dashed')
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles, ['Normal', 'Obese'])
    ax.set(xlabel='Mass, g');
    return ax

total_len = 40
x, y = generate_data(total_len=total_len)
ax = plot_data(x, y, threshold=21.5, total_len=total_len)

Теперь пользуясь нашим простым критерием, попробуем классифицировать каких-то новых мышей

In [ ]:
x_test = np.random.uniform(14, 30, 5)

def classify(x, threshold=21.5):
    y = np.zeros_like(x)
    y[x > threshold] = 1
    return y

total_len = 40
threshold = 21.5
x, y = generate_data(total_len=total_len)
ax = plot_data(x, y, threshold=threshold, total_len=total_len)
ax = plot_data(x_test, classify(x_test, threshold), total_len=total_len, s=300)

Но что если наши мыши находятся тут?


In [ ]:
x_test = np.array([21.45, 22.5])

total_len = 40
x, y = generate_data(total_len=total_len)
ax = plot_data(x, y, total_len=total_len)
ax = plot_data(x_test, classify(x_test), threshold=21.5, total_len=total_len, s=300)

С точки зрения нашего классификатора - все четко. Больше порогового значения - значит перевес, меньше - значит нормальные. Но с точки зрения здравого смысла, логичнее было бы классифицировать обоих мышей как нормальных, так как они значительно ближе к нормальным, чем к ожиревшим.

Вооружившись этим новым знанием, попробуем классифицировать наших отъевшихся мышек по-умному. Возьмем крайние точки в каждом кластере. И в качестве порогового значения будем использовать среднее между ними

In [ ]:
x, y = generate_data(total_len=total_len)
normal_limit = x[y==0].max() # extreme point for 'normal'
obese_limit = x[y==1].min() # extreme point for 'obese'

threshold = np.mean([normal_limit, obese_limit]) # separated with mean value 

x_test = np.array([21.5, 23])
ax = plot_data(x, y, total_len=total_len, threshold=threshold, margin=[normal_limit, obese_limit])
ax = plot_data(x_test, classify(x_test, threshold=threshold), total_len=total_len, s=300, threshold=threshold)

Мы можем посчитать, насколько наша мышь близка к тому, чтобы оказаться в другом классе. Такое расстояние называется **margin**. И оно считается как $\mathrm{margin} = |\mathrm{threshold} - \mathrm{observation}|$

In [ ]:
margins = np.abs(x_test - threshold)
print(margins)

Соответственно, если мы посчитаем margins для наших крайних точек `normal_limit` и `obese_limit`, мы найдем самое большое возможное значение margin для нашего классификатора

In [ ]:
margin_0 = np.abs(normal_limit - threshold)
margin_1 = np.abs(obese_limit - threshold)
print(margin_0, margin_1)

Такой классификатор, мы называем **Maximum Margin Classifier**. Он хорошо работает в случае, когда все данные размечены аккуратно. Теперь рассмотрим более реалистичный пример, где что-то пошло не так

In [ ]:
def generate_realistic_data(total_len=40):
    x = np.hstack([np.random.uniform(14, 21, total_len // 2), np.random.uniform(24, 33, total_len // 2)])
    y = np.hstack([np.zeros(total_len // 2), np.ones(total_len // 2)])
    indx = np.where(x == x[y==1].min())[0]
    y[indx] = 0
    s = np.ones_like(x) * 50
    s[indx] = 300
    return x, y, s

total_len = 40
x, y, s = generate_realistic_data(total_len=total_len)
ax = plot_data(x, y, total_len=total_len, s=s)

В таком случае, наш **Maximum Margin Classifier** работать не будет. Исходя из этого, мы можем прийти к выводу, что наш классификатор очень чувствителен к выбросам. Давайте подумаем можно ли это как-то исправить?

Например, мы можем разрешить нашему классификатору ошибаться. Если мы будем использовать в качестве порогового значения не самое крайнее, а следующее за ним - мы промажем в классификации конкретно этой странной точки. Но в целом будем правы. Margin определенный таким образом, называется **Soft margin**.

In [ ]:
total_len = 40
x, y, s = generate_realistic_data(total_len=total_len)

normal_limit = np.sort(x[y==0])[-2]
obese_limit = np.sort(x[y==1])[1]

threshold = np.mean([normal_limit, obese_limit])

x_test = np.array([20.5, 25])
ax = plot_data(x, y, total_len=total_len, threshold=threshold)
ax = plot_data(x_test, classify(x_test, threshold=threshold), total_len=total_len, 
               s=300, threshold=threshold, margin=[normal_limit, obese_limit])

Но почему мы решили взять именно следующее значение? Почему не через 2? Откуда мы знаем, что это лучший из возможных вариантов? А ни откуда не знаем. Чтобы узнать какой из margins лучше, нам стоит численно это проверить и посчитать, сколько раз мы ошибемся, если возьмем в качестве порогового значения между каждой парой точек. Для этого мы вновь воспользуемся *кросс-валидацией*.

In [ ]:
total_len = 40
x, y, s = generate_realistic_data(total_len=total_len)

indxs = []
accuracies = []
thresholds = []

x_test, y_test = generate_data(total_len=total_len)

for i in range(total_len // 2):
    for j in range(total_len // 2 - 1):
        normal_limit = np.sort(x[y==0])[-i]
        obese_limit = np.sort(x[y==1])[j]

        threshold = np.mean([normal_limit, obese_limit])
        
        y_pred = classify(x_test, threshold=threshold)
        accuracy = np.mean(y_test == y_pred)
        indxs.append((-i, j))
        accuracies.append(accuracy)
        thresholds.append(threshold)

print(f'Accuracy = {np.max(accuracies) * 100}%')
best_index = indxs[np.argmax(accuracies)][1]
print(f'Best indexes', best_index)
best_treshold = thresholds[np.argmax(accuracies)]
print('Best treshhold value %.2f'% best_treshold)

ax = plot_data(x, y, total_len=total_len, threshold=best_treshold)
ax = plot_data(x_test, y_test, 
               total_len=total_len, 
               s=200, 
               threshold=best_treshold)

Когда для классификатора используется **Soft Margin** - такой классификатор называют **Soft Margin Classifier** или по-другому - **Support Vector Classifier**. По сути это уже SVM. 

### 2D классификация

Теперь рассмотрим пример, где мы измерили не только вес мышей, но и их длину от хвоста до носа. Мы можем вновь применить наш метод Support Vector Classifier, и теперь классы разделяет не одно пороговое значение (по сути, точка), а линия.

In [ ]:
from sklearn.datasets import make_blobs
from sklearn import svm

def generate_2d_data(total_len=40):
    x, y = make_blobs(n_samples=total_len, centers=2, random_state=42)
    x[:, 0] += 10 
    x[:, 1] += 20 
    return x, y

def plot_data(x, y, total_len=40, s=50, threshold=21.5):
    ax = sns.scatterplot(x=x[:, 0], y=x[:, 1], hue=y, s=s)
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles, ['Normal', 'Obese'])
    ax.set(xlabel='Mass, g', ylabel='Length, cm');
    return ax

total_len = 40
x, y = generate_2d_data(total_len=total_len)
ax = plot_data(x, y, total_len=total_len)

# Code for illustration, later we will understand how it works
# fit the model, don't regularize for illustration purposes
clf = svm.SVC(kernel='linear', C=1000)
clf.fit(x, y)

# plot the decision function
ax = plt.gca()
xlim = ax.get_xlim()
ylim = ax.get_ylim()

# create grid to evaluate model
xx = np.linspace(xlim[0], xlim[1], 30)
yy = np.linspace(ylim[0], ylim[1], 30)
YY, XX = np.meshgrid(yy, xx)
xy = np.vstack([XX.ravel(), YY.ravel()]).T
Z = clf.decision_function(xy).reshape(XX.shape)

# plot decision boundary and margins
ax.contour(XX, YY, Z, colors='k', levels=[-1, 0, 1], alpha=0.5,
           linestyles=['--', '-', '--'])
# plot support vectors
ax.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1], s=100,
           linewidth=1, facecolors='none', edgecolors='k')
plt.show()

### 3D классификация

Если мы добавим еще одно измерение - возраст, мы обнаружим, что наши данные стали трехмерными, а разделяет их теперь не линия, а плоскость

In [ ]:
def generate_3d_data(total_len=40):
    x, y = make_blobs(n_samples=total_len, centers=2, random_state=42, n_features=3)
    x[:,0] += 10 
    x[:,1] += 20 
    x[:,2] += 10
    return x, y

def plot_data(x, y, total_len=40, s=50, threshold=21.5):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(xs=x[:, 0], ys=x[:, 1], zs=x[:, 2], c=y, s=s, cmap='Set1')
    # plot the decision function
    ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

    # create grid to evaluate model
    xx = np.linspace(xlim[0], xlim[1], 30)
    yy = np.linspace(ylim[0], ylim[1], 30)
    YY, XX = np.meshgrid(yy, xx)
    ax.plot_surface(XX, YY, XX * YY * 0.2, alpha=0.2)
    handles, labels  =  ax.get_legend_handles_labels()
    ax.legend(handles, ['Normal', 'Obese'])
    ax.set(xlabel='Mass, g', ylabel='Length, cm', zlabel='Age, days');
    return ax

total_len = 40
x, y = generate_3d_data(total_len=total_len)
ax = plot_data(x, y, total_len=total_len)


Соответственно, если бы у нас было 4 измерения и больше (например: вес, длинна, возраст, кровяное давление), то многомерная плоскость, которая бы разделяла наши классы, называлась бы **гиперплоскость** (рисовать мы ее, конечно же, не будем). Чисто технически, и точка, и линия - тоже гиперплоскости. Но все же гиперплоскостью принято называть то, что нельзя нарисовать на бумаге.

### SVM

Данные не всегда разделяются так хорошо как в случае нашего мышиного датасета. Например, рассмотрим следующее: у нас есть данные по дозировке лекарства и 2 класса - пациенты, которые поправились, и те, которым лучше не стало

In [ ]:
def generate_patients_data(total_len=40):
    x = np.random.uniform(0, 50, total_len)
    y = np.zeros_like(x)
    y[(x > 15) & (x < 35)] = 1
    return x, y

def plot_data(x, y, total_len=40, s=50):
    ax = sns.scatterplot(x=x, y=np.zeros(len(x)), hue=y, s=s)

    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles, ['Recover', 'Sick'])
    ax.set(xlabel='dose, mg');
    return ax

total_len = 40
x, y = generate_patients_data(total_len=total_len)
ax = plot_data(x, y, total_len=total_len)

Соответственно мы не можем найти такое пороговое значение, которое будет разделять наши классы на больных и здоровых, а, следовательно, и Support Vector Classifier работать тоже не будет.  Для начала, давайте преобразуем наши данные таким образом, что бы они стали 2-х мерными. В качестве значений по оси Y будем использовать дозу возведенную в квадрат (**доза**$^2$).

In [ ]:
def plot_data(x, y, total_len=40, s=50):
    ax = sns.scatterplot(x=x[0, :], y=x[1, :], hue=y, s=s)
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles, ['Recover', 'Sick'])
    ax.set(xlabel='Dose, mg');
    ax.set(ylabel='Dose$^2$');
    return ax

total_len = 40
x_1, y = generate_patients_data(total_len=total_len)
x_2 = x_1 ** 2
x = np.vstack([x_1, x_2])

plot_data(x, y, total_len=40, s=50)
plt.show()

Теперь мы можем вновь использовать Support Vector Classifier для классификации

In [ ]:
plot_data(x, y, total_len=40, s=50)

x_arr = np.linspace(0, 50, 50)
xs = [x[0, :][y==1].min(), x[0, :][y==1].max()]
ys = [x[1, :][y==1].min(), x[1, :][y==1].max()]

# Calculate the coefficients.
coefficients = np.polyfit(xs, ys, 1)

# Let's compute the values of the line...
polynomial = np.poly1d(coefficients)
y_axis = polynomial(x_arr)

# ...and plot the points and the line
plt.plot(x_arr, y_axis, 'r--')
plt.show()

Но тут возникает резонный вопрос - почему мы решили возвести в квадрат? Почему не в куб? Или наоборот не извлечь корень? Как нам решить какое преобразование использовать?

И у нас есть **вторая проблема** - а если перейти надо в пространство очень большой размерности? В этом случае наши данные очень сильно увеличатся в размере.

Комбинация двух проблем дает нам **много радости** - надо перебирать большое число возможных пространств большей размерности



Однако основная фишка Support Vector Machine состоит в том, что внутри он работает на скалярных произведениях. И можно эти скалярные произведения считать, **не переходя в пространство большей размерности**


Для этого SVM использует **Kernel Function**. 

Kernel Function может, например, быть полиномом (**Polynomial Kernel Function**), который имеет параметр $d$ - сколько размерностей выбрать. 

<img src="https://edunet.kea.su/repo/EduNet-content/L02/out/svm_kernel_function.png" width="700">

Примеры ядер :

* $k(x_i, x_j) = (<x_i, x_j> + c)^d, с, d \in \mathbb{R}$ - полиномиальное ядро, считает расстояние между объектами в пространстве размерности d

* $k(x_i, x_j) = \frac{1}{z} e^{-\frac{h(x_i, x_j)^2}{h}}$ - радиальная базисная функция RBF


Таким образом, в случае SVM можно легко перебрать много таких пространств на кроссвалидации и выбрать более удобное. 

Более того, SVM может проверять пространства признаков бесконечного размера, если для такого пространства существует kernel function. Для решения практических задач иногда такие бесконечномерные пространства признаков могут оказаться удобными. Широко применяемое на практике RFB ядро как раз соответствует такому случаю бесконечномерного пространства признаков.


### Регрессия

#### Пример линейной регрессии

Рассмотрим несколько другую задачу. Пусть мы хотим построить модель предсказания для величины успеваемости студента и хотим это делать на основе информации о величине потраченного им на изучения материала количества времени в часах. Это пример простейшей задачи линейной регрессии.

**Регрессией** (регрессионным анализом) называется набор статистических методов, использующихся для определения характера связи между одной зависимой переменной (традиционно обозначаемой $Y$ и называемой также "откликом", "результатом" или "лейблом") и одной или рядом других независимых переменных (традиционно обозначаемых $X$ и называемых также "предикторами", "ковариатами" или "признаками").

Загрузим датасет. Мы должны увидеть, что он содержит два числовых признака - часы и результаты.

In [ ]:
import pandas as pd

dataset = pd.read_csv("https://edunet.kea.su/repo/EduNet-web_dependencies/L02/student_scores.csv")
print(dataset.shape)
dataset.head()

Построим график зависимости одного от другого, а так же отобразим распределения каждой из переменных

In [ ]:
sns.jointplot(data=dataset, x="Scores", y="Hours")
plt.show()

Разделим наши данные на train и test

In [ ]:
from sklearn.model_selection import train_test_split

x = dataset.iloc[:, :-1].values # column Hours
y = dataset.iloc[:, 1].values # column Score

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42
)

Теперь создадим модель для линейной регрессии. Чтобы не писать с нуля, воспользуемся готовой моделью из библиотеки `sklearn`

In [ ]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression() 

И обучим ее

In [ ]:
regressor.fit(x_train, y_train) 

Посмотрим, что получилось

In [ ]:
x_train.shape

In [ ]:
x_points = np.linspace(min(x_train), max(x_train), 100)  # 100 dots at min to max
y_pred = regressor.predict(x_points)  

plt.figure(figsize=(8, 5))
plt.plot(x_train, y_train, "o", label="Scores")
plt.plot(x_points, y_pred, label="y = %.2fx+%.2f" % (regressor.coef_[0], regressor.intercept_ ))
plt.title("Hours vs Percentage", size=15)
plt.xlabel("Hours Studied", size=15)
plt.ylabel("Percentage Score", size=15)
plt.legend()
plt.show()

Теперь сделаем предсказание для тестовой выборки

In [ ]:
y_pred = regressor.predict(x_test)  

x_points = np.linspace(min(x_test), max(x_test), 100)  
y_pred = regressor.predict(x_points) 

plt.figure(figsize=(8, 5))
plt.plot(x_test, y_test, "o", label="Scores")
plt.plot(x_points, y_pred, label="y = %.2fx+%.2f" % (regressor.coef_[0], regressor.intercept_ ))
plt.title("Hours vs Percentage", size=15)
plt.xlabel("Hours Studied", size=15)
plt.ylabel("Percentage Score", size=15)
plt.legend()
plt.show()

Выглядит неплохо

Посчитаем метрики для наших значений

In [ ]:
from sklearn import metrics

y_pred = regressor.predict(x_test)

print("Mean Absolute Error: %9.2f" % metrics.mean_absolute_error(y_test, y_pred))
print("Mean Squared Error: %10.2f" % metrics.mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error: %5.2f" % np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

#### Геометрическая интерпретация 

Теперь, когда мы разобрались с тем, что такое регрессия и с чем ее едят, вернемся к нашим картинкам. Как можно применить регрессию для классификации?

Предположим у нас есть только 2 класса. Как можно использовать регрессию для того, чтобы определить относится ли изображение к классу 0 или к классу 1? В упрощенном варианте, задача будет состоять в том, чтобы провести разделяющую плоскость (прямую) между 2-мя классами. Например, мы можем провести прямую через 0. 


<img src ="https://edunet.kea.su/repo/EduNet-content/L02/out/regression_for_classification_imgs.png" width="270">





Рассмотрим другую ситуацию, в этом случае, мы не можем просто провести прямую через 0. Но можем отступить от 0 на какое-то расстояние и провести ее там. Вспомним, что уравнение прямой это $y=wx+b$, где $b$ - это смещение (*bias*). Соответственно если b != 0, то прямая через 0 проходить не будет, а будет проходить через значение b.

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/out/regression_for_classification_add_bias.png" width="270">

[Linear Classification Loss Visualization
](http://vision.stanford.edu/teaching/cs231n-demos/linear-classify/)


Если у нас есть несколько классов (несколько шаблонов), мы можем для каждого из них посчитать уравнение $y_{i} = w_{i}x_{i}+b_{i}$.

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/out/regression_for_classification_add_bias_add_multiclasses.jpg" width="400">







На картинке нас интересуют 3 класса. Соответственно, мы можем записать систему линейных уравнений:

\begin{aligned}
y_{0} = w_{0}x_{0} + b_{0} \\
y_{1} = w_{1}x_{1} + b_{1} \\
y_{2} = w_{2}x_{2} + b_{2} \\
\end{aligned}

#### Добавление смещения

Мы их можем собрать в матрицу, тогда получится следующее:

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/out/scalar_product_add_bias.png" width="750">

У нас есть матрица коэффициентов, которые мы каким-то образом подобрали, пока ещё не понятно как. Есть вектор $x$, соответствующий изображению. 

Мы умножаем вектор на матрицу, получаем нашу гиперплоскость для четырехмерного пространства в данном случае. Чтобы оно не лежало в 0, мы должны добавить смещение. И мы можем сделать это после, но можно взять и этот вектор смещения (вектор **b**) просто приписать к матрице **W**.

Что будет выходом такой конструкции? Мы умножили матрицу весов на наш вектор, соответствующий изображению, получили некоторый отклик. По этому отклику мы так же, как и при реализации метода ближайшего соседа можем судить: если он больше остальных, то мы предполагаем, что это кошка.


In [ ]:
img = np.array([56, 231, 24, 2])
w_cat = np.array([0.2, -0.5, 0.1, 2.0])
print("Image ", img)
print("Weights ", w_cat)
print("img * w_cat ", img * w_cat)
print("sum ", (img * w_cat).sum())
print("Add bias ", (img * w_cat).sum() + 1.1)

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/out/img_to_function_get_scores.png" width="600">


Собирая все вместе, получаем какое-то компактное представление, что у нас есть некоторая функция, на вход которой мы подаем изображение, и у нее есть параметры (веса). Пока происходит просто умножение вектора на матрицу, в дальнейшем это может быть что-то более сложное, функция будет представлять какую-то более сложную модель. А на выходе (для классификатора) мы получаем числа, которые интерпретируют уверенность модели в том, что изображение принадлежит к определенному классу.


<img src ="https://edunet.kea.su/repo/EduNet-content/L02/out/input_img_scalar_product_add_bias_get_scores.png" width="450">



Соответственно, эти коэффициенты, которые являются весами модели, надо каким-то образом подбирать. Но прежде чем подбирать коэффициенты, давайте определимся со следующим: как мы будем понимать, что модель работает хорошо или плохо? Вывод модели представляет собой просто некоторый набор чисел. Но как эти числа следует правильно интерпретировать? Рассмотрим этот вопрос в следующем разделе.

# Функция потерь SVM

Давайте рассмотрим как метод опорных векторов (SVM) работает на практике. Мы можем применить метод опорных векторов в том числе и к изображениям -- достаточно просто вытянуть изображение из тензора формата $(\text{C}, \text{H}, \text{W})$ в $(\text{C} \cdot \text{H} \cdot \text{W})$-мерный вектор. Применим SVM к нескольким изображениям из датасета CIFAR10:

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/out/model_predicted_scores_for_10_classes.png" width="550">

По аналогии с тем, что мы уже делали, мы можем сравнивать отклик на ключевой класс  (про который нам известно, что он на изображении, так как у нас есть метка этого класса) с остальными. Соответственно, мы подали изображение кошки и получили на выход вектор. Чем больше значение, тем больше вероятность того, что, по мнению модели, на изображении этот класс. Для кошки в данном случае это значение 2.9. Хорошо это или плохо? Нельзя сказать, пока мы не проанализировали остальную часть вектора. Если бы мы могли посмотреть на все значения в векторе, мы бы увидели, что есть значения больше, то есть в данном случае модель считает, что это собака, а не кошка, потому что для собаки значение максимально. 

На основании этого можно построить некоторую оценку. Давайте смотреть на разницу правильного класса с неправильными. В зависимости от того, насколько уверенность в кошке будет больше остальных, настолько будет наша модель хорошей.

Но поскольку нам важна не работа модели на конкретном изображении, а важно оценить ее работу в целом, то эту операцию нужно проделать либо для всего датасета, либо для некоторой выборки, которую мы подаем на вход и подсчитываем средний показатель. Этот показатель (насколько хорошо работает модель), называется функцией потерь, или **loss функцией**. Называется она так, потому что она показывает не то, насколько хорошо работает модель, а то, насколько плохо. 

Дальше будет понятно, почему так удобнее (разница только в знаке). Как это посчитать для всего датасета?

Мы каким-то образом считаем loss для конкретного изображения, потом усредняем по всем изображениям.

Дано: 3 учебных примера, 3 класса. При некотором W баллы f (a, W) = Wx равны:

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/out/model_predicted_scores_for_3_classes.png" width="450">

Функция потерь показывает, насколько хорош наш текущий классификатор.

Дан датасет примеров:

$\begin{Bmatrix} (x_i,y_i)  \end{Bmatrix}_{i=1}^N 	$

Где **$x_i$** изображение и **$y_i$** метка (число).

Потери по набору данных — это среднее значение потерь для примеров:

$ L = {1 \over N}\sum_iL_i(f(x_i,W),y_i)$


Построим функцию потерь для одного примера $L_i(f(x_i,W),y_i)$:

1. Вычислим вектор значений прогнозов классификатора $s = f(x_i, W)$.
1. Для всех примеров рассмотрим разницу между оценкой на истинной категории и всеми оценками классификатора для неправильных категорий: $s_{y_i} - s_j$ для $j \neq y_i$. 
1. Если получившаяся разница положительная и превышает некоторое пороговое значение («зазор»), которое мы установим равным $1$, то будем считать, что категория $j$ не мешает модели верно классифицировать входной объект, припишем категории $j$ нулевой вклад в $L_i(f(x_i,W),y_i)$.
1. Если получившаяся разница не превосходит установленного нами единичного «зазора», то мы будем считать что ответ классификатора $s_j$ в категории $j$ мешает верной классификации входного объекта. В этом случае припишем для категории $j$ аддитивный вклад в $L_i(f(x_i,W),y_i)$ равный $s_j-s_{y_i}+1$.

Описанную процедуру гораздо проще записать в виде формулы:


$L_i = \sum_{j\neq y_i}\begin{cases}
  0,  & \mbox{если } s_{y_i}\geq s_j+1\mbox{} \\
  s_j-s_{y_i}+1, & \mbox{если наоборот, то} \mbox{}
\end{cases}$

$=\sum_{j\neq y_i}max(0,s_j-s_{y_i}+1)$




## Вычисление функции потерь SVM 


 Не самая мощная, но достаточно интуитивно понятная loss функция –  SVM loss. 
 
Логика такая: если у нас уверенность модели в правильном классе большая, то модель работает хорошо и loss для данного конкретного примера должен быть равен нулю. Если есть класс, в котором модель уверена больше, чем в правильном, то loss должен быть не равен нулю, а отображать какую-то разницу, поскольку модель сильно ошиблась. При этом есть ещё одно соображение: что будет, если на выходе у правильного и ошибочного класса будут примерно равные веса? То есть, например, у кошки было бы 3.2, а у машины не 5.2, а 3.1. В этом случае ошибки нет, но понятно, что при небольшом изменении в данных (просто шум) скорее всего она появится.
 


То есть модель плохо отличает эти классы. Поэтому мы и вводили некоторый зазор, который должен быть между правильным и неправильным ответом. 

Посмотрим на изображение снизу. У нас есть два класса: фиолетовые треугольники и синие квадраты, разделенные зазором. Также можем увидеть желтые треугольники и квадраты - это ошибочно распознанные классы.

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/out/svm_decision_boundary.png" width="300">


И тоже учитывать его в loss функции: то есть сравнивать его результат для правильного класса не с чистым выходом для другого, а добавить к нему некоторую дельту (в данном случае – 1(единица)). Смотрим: если разница больше 0, то модель работает хорошо и loss равно нулю. Если нет, то мы возвращаем эту разницу, и loss будет складываться из этих индивидуальных разниц.

$L_i = \sum_{j\neq y_i}\begin{cases}
  0,  & \mbox{если } s_{y_i}\geq s_j+1\mbox{} \\
  s_j-s_{y_i}+1, & \mbox{если наоборот, то} \mbox{}
\end{cases}$

$=\sum_{j\neq y_i}max(0,s_j-s_{y_i}+1)$

Ниже пример того, как считается loss.


Считаем функцию потерь для 1-ого изображения:

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/out/compute_loss_use_model_scores_for_1_example.png" width="650">

Также считаем потери для 2-ого и 3-его изображения:

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/out/compute_loss_use_model_scores_for_2_and_3_examples.png" >

Значения losses получились следующие:

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/out/summary_losses_for_3_examples.png" width="450">

Считаем среднее значение loss для всего датасета:

$ L = {1 \over N}\sum_{i=1}^N L_i$

$L={2.9 + 0 + 12.9 \over 3} = 5.27$


SVM loss

$L_i=\sum_{j\neq y_i}max(0,s_j-s_{y_i}+1)$


Как записать это в коде:

In [ ]:
import os
from IPython.display import clear_output

file_exists = os.path.exists("/content/cifar-10-batches-py")
if file_exists == False:
    !wget https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
    !tar -xzf cifar-10-python.tar.gz
    
clear_output()

In [ ]:
import pickle
import numpy as np 

def unpickle(file):
    with open(file, "rb") as fo:
        dict = pickle.load(fo, encoding="bytes")
    return dict

x_train = np.zeros((0, 3072))
y_train = np.array([])
for i in range(1, 6):
    raw = unpickle(f"/content/cifar-10-batches-py/data_batch_{i}")
    x_train = np.append(x_train, np.array(raw[b"data"]), axis=0)
    y_train = np.append(y_train, np.array(raw[b"labels"]), axis=0)

test = unpickle("/content/cifar-10-batches-py/test_batch")
x_test = np.array(test[b"data"])
y_test = np.array(test[b"labels"])

labels_eng = [
    "Airplane",
    "Car",
    "Bird",
    "Cat",
    "Deer",
    "Dog",
    "Frog",
    "Horse",
    "Ship",
    "Truck",
]

print(x_train.shape)
print(x_test.shape)

In [ ]:
W = np.random.randn(10, 3072) * 0.0001 # random weights

In [ ]:
assa = np.maximum(0, W.dot(x_train[0]) - W.dot(x_train[0])[6] + 1)
assa[6] = 0
np.sum(assa)

In [ ]:
def Li_svm(x, y, W):
    y = int(y)  # to use like a index
    scores = W.dot(x)  
    margins = np.maximum(0, scores - scores[y] + 1)  # loss
    margins[y] = 0  
    return np.sum(margins)  


L0 = Li_svm(x_train[0], y_train[0], W)
print("Loss on first image", L0)

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/out/losses_at_set_of_data.png" width="900">

Собираем все вместе: считаем loss для всего датасета, усредняя ее.


Как будет выглядеть график этой SVM-loss?

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/out/svm_plot.gif" width="400">


Функция потерь для ситуации, когда зазор большой, будет равна 0. Она будет меняться только тогда, когда у нас есть ошибка. Причем будет меняться линейно.

Как мы можем использовать эту loss функцию? Допустим, мы знаем, что у нас модель будет работать не очень хорошо (выдает большую ошибку). Можно попытаться найти минимум этой loss функции, то есть подобрать такое W, когда loss функция обратится в 0  (в данном случае), а в общем случае (если какое-то количество данных нельзя обратить в 0), мы можем попытаться найти ее минимум.



# Обновления весов методом градиентного спуска

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/out/backpropagation_weight_optimization.png" width="750">

Идея в следующем: у нас есть некоторая поверхность, на которой будет точка минимума функции. Мы должны обновлять веса таким образом, чтобы смещаться в сторону этой точки. Loss показывает, как возрастает функция потерь. Так как потери - это плохо, мы должны их минимизировать, то есть мы должны уменьшать веса в сторону, обратную росту этой функции.

Если переходить на случай n-мерного (в данном случае трехмерного) пространства, здесь достаточно очевидна аналогия с поверхностью: у нас есть поверхность земли, которая описывается координатами $x, y, z$. Если мы движемся по этой поверхности, высота $(z)$ будет зависеть от $x, y$. 

$x, y$ - это координаты. Мы можем записать их как вектор. Наша функция будет работать с вектором координат и выдавать скаляр, третью координату. Если в качестве координат мы будем использовать веса нашей модели, а в качестве $z$ - loss, то аналогия станет полной. Наша задача сведется к тому, чтобы найти такой набор весов, при котором значение функции будет минимально. То есть мы окажемся в каком-то минимуме, где ошибка будет минимально возможна для этих данных.

Для того чтобы предположить, где он может находиться, надо понимать, в какую сторону функция растет, а в какую - убывает. Для этого существует производная.

Поскольку у нас функция от нескольких переменных, если мы будем брать от нее производную, у нас получится вектор частных производных, то есть градиент этой функции, который будет показывать, как она меняется в каждом направлении.

## Градиент функции потерь


Как известно из курса математического анализа, производная вещественной функции одной переменной характеризует мгновенную скорость изменения значений данной функции. Если в какой-то точке производная положительна, то в этой окрестности функция может возрастать. В противном случае, если производная отрицательна, функция может убывать. Аналогичный смысл несёт градиент для вещественной скалярной функции многих переменных.

Возьмём гладкую скалярная функцию многих вещественных переменных (таковой, например, является функция потерь) и вычислим производные по каждой из её переменных, считая остальные переменные зафиксированными (так называемые частные производные). Запишем получившиеся частные производные в виде вектора-столбца.
Каждый из компонентов такого вектора будет характеризовать мгновенную скорость изменения функции вдоль направления одной из переменных. Если же посмотреть на такой вектор в целом, то он будет направлен в сторону наибольшего роста нашей скалярной функции, а его длина будет характеризовать скорость такого роста.

Например, если в качестве скалярной функции $\varphi$ взять высоту поверхности земли над уровнем моря,то $\text{grad} \varphi$ в каждой точке земной поверхности будет направлен в сторону самого крутого подъема и будет своей длиной указывать крутизну склона.     

Так, если $\varphi = \varphi(x_1 \dots x_n)$  — функция $n$ переменных, то её градиентом называется $n$-мерный вектор:
$$\left(\frac{\partial\varphi}{\partial x_1},\dots,\frac{\partial\varphi}{\partial x_n}\right)^T$$

Нас будет интересовать градиент функции потерь $L$, взятый по весам модели $W$:


$W$ - матрица(вектор) весов

$L$ - функция потерь (скаляр)

$\partial W = W_2 - W_1$

$\partial L = L_2 - L_1$


$\frac{\partial L}{\partial W}=\begin{bmatrix}
\frac{\partial L}{\partial W_1} \\
\frac{\partial L}{\partial W_2} \\
... \\
\frac{\partial L}{\partial W_n}
\end{bmatrix}$


Наша задача будет сводиться к тому, что мы будем искать градиент loss функции по весам, которые будут состоять из производной по каждому направлению. Поскольку у нас здесь числа, можно считать этот градиент численно.

## Численный расчет производной

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/out/gradient_descent_analytical_calculation.png" width="650">



Посчитаем градиент приближенно, воспользовавшись определением (в формуле аргумент обозначен как $x$, у нас же аргументом будет $W$):
На нулевом шаге у нас есть $W_0$ найдем $L_0 = Loss(f(W_0,x))$
Прибавим к первому элементу $W_0$ небольшую величину  $h$ = 0.0001 и получим новую матрицу весов $W_1$ отличающуюся от $W_0$ на один единственный элемент.

Найдем Loss от $\frac {W_1}  {L_1} = Loss(f(W_1,x))$
По определению производной $\frac {dL}{W_0} = \frac{( L_1 - L_0 )}   {h}$

Повторяя этот процесс для каждого элемента из $W$, найдем вектор частных производных, то есть градиент $\frac{dL}{dW}$.

Плюсы:
Это просто. 

Проблемы:

1. Это очень долго, нам придется заново искать значение loss функции для каждого $W_i$.

2. Это неточно, так как по определению приращение $h$ бесконечно мало, а мы используем конкретное пусть и небольшое число. И если мы сделаем его слишком маленьким, то столкнемся с ошибками связанными с округлением в памяти компьютера.


Поэтому данный метод может использоваться как проверочный. 
А на практике вместо него используется **аналитический расчет градиента**.


## Аналитический расчет производной от функции потерь SVM

### Простые производные 

$$x' = \frac {\delta x} {\delta x} = 1$$ 

$$(x^2)' = \frac {\delta x^2} {\delta x} = 2x$$

$$(\log x)'  = \frac {\delta \log x} {\delta x} = \frac 1 x $$

$$(e^x)'  = \frac {\delta e^x} {\delta x} = e^x $$

$$\frac {\delta cf(x)} {\delta x}= c \cdot \frac {\delta f(x)} {\delta x}$$

$$\frac {\delta f(x) + c} {\delta x}= \frac {\delta f(x)} {\delta x}$$

$c$ - константа, не зависящая от $x$

$$ \frac {\delta [f(x) + g(x)]} {\delta x} = \frac {\delta f(x)} {\delta x}  + \frac {\delta g(x)} {\delta x} $$ 


$$\frac {\delta (x^2 + y^3)} {\delta x} = 2x  $$
так как $y$ по отношению к $x$ - константа и мы меняем только $x$

$$\frac {\delta (x^2 + y^3)} {\delta y} = 3y^2  $$
так как $x$ по отношению к $y$ - константа и мы меняем только $y$

$$(e^y)'  = \frac {\delta e^y} {\delta x} = 0 $$

### Chain-rule

Производная функции $f(g)$:

$$\frac {\delta f} {\delta g}$$

Пусть $g$  на самом деле не просто переменная, а зависит от $h$. Тогда производная от $f$ по $g$ **не меняется**, а производная $f$ по $h$ запишется следующим образом:


$$\frac {\delta f(g(h))} {\delta h} = \frac {\delta f} {\delta g} \frac {\delta g} {\delta h}$$

Пусть теперь $h$ зависит от $x$. Все аналогично


$$\frac {\delta f(g(h(x)))} {\delta x} = \frac {\delta f} {\delta g} \frac {\delta g} {\delta h} \frac {\delta h} {\delta x}$$

Так можно делать до бесконечности, находя производную сколь угодно сложной функции. И, что важно - мы можем считать градиенты частями - посчитать сначала $f$ по $g$, потом $g$ по $h$....

$$\frac {\delta log(x^2 + 5)} {\delta x}$$

$$h = x^2 + 5$$

$$\frac {\delta log(x^2 + 5)} {\delta x} = \frac {\delta log(h)} {\delta h} \frac {\delta h} {\delta x}$$ 

$$\frac {\delta log(h)} {\delta h} = \frac 1 h$$

$$\frac  {\delta h} {\delta x} = 2x $$

$$\frac {\delta log(x^2 + 5)} {\delta x} = \frac 1 {x^2 + 5} \cdot 2x = \frac {2x} {x^2 + 5}$$


###  Часть MSE-loss

$$loss = (y - \hat{y})^2 $$

$$\hat{y} = wx + b $$

$$ \frac {\delta loss} {\delta w} = \frac {\delta loss} {\delta \hat{y}} \cdot \frac {\delta \hat{y}} {\delta w} $$

$$ \frac {\delta loss} {\delta \hat{y}} = \frac {\delta (y - \hat{y})^2 } {\delta {(y - \hat{y})}} \frac {\delta y - \hat{y}} {\delta \hat{y}} = 2(y-\hat{y}) \cdot -1 = 2 (\hat{y} - y)$$

$$ \frac {\delta \hat{y}} {\delta w} = \frac {\delta wx + b} {\delta w} = x$$

$$ \frac {\delta loss} {\delta w} = 2 x \cdot (\hat{y} - y) $$

### MSE-loss
$$MSE = \frac 1 N \sum_i(y_i - \hat{y_i})^2 $$

$y_i$ - константы
$\hat{y_i}$ - не являются функциями друг от друга 
$$\hat{y} = wx_i + b $$

$$\frac {\delta MSE} {\delta w} = \frac 1 N \sum \frac {\delta (y_i - \hat{y_i}) ^2} {\delta \hat{y_i}} \frac {\delta \hat{y_i}} {\delta w}$$




### Часть MAE-Loss

$$loss = |y - \hat{y}| $$

$$\hat{y} = wx + b $$

$$ \frac {\delta loss} {\delta w} = \frac {\delta loss} {\delta \hat{y}} \cdot \frac {\delta \hat{y}} {\delta w} $$

$$ \frac {\delta \hat{y}} {\delta w} = \frac {\delta wx + b} {\delta w} = x$$

$$ \frac {\delta loss} {\delta \hat{y}} = \frac {\delta |y - \hat{y}|) } {\delta {(y - \hat{y})}} \frac {\delta y - \hat{y}} {\delta \hat{y}} = \frac {\delta |y - \hat{y}|) } {\delta {(y - \hat{y})}} \cdot -1 = - \frac {\delta |y - \hat{y}|) } {\delta {(y - \hat{y})}}$$

Строго говоря, у модуля не существует производной в 0. 







In [ ]:
import matplotlib.pyplot as plt

x = [i for i in range(-5, 6)]
y = [abs(i) for i in range(-5, 6)]

plt.figure(figsize=(8, 5))
plt.plot(x, y, label="y = |x|")
plt.title("y = |x|", size=20)
plt.legend()
plt.show()

Но мы можем сказать, что в этой точке производная равна 0.
Если аргумент модуля меньше 0, то производная будет -1.

Если больше +1

$$ \frac {\delta loss} {\delta \hat{y}} = \frac {\delta |y - \hat{y}| } {\delta {(y - \hat{y})}} \frac {\delta y - \hat{y}} {\delta \hat{y}} = \frac {\delta |y - \hat{y}| } {\delta {(y - \hat{y})}} \cdot -1 = - \frac {\delta |y - \hat{y}| } {\delta {(y - \hat{y})}} = - sign(y - \hat{y}) =  sign(\hat{y} - y)$$

<br>
<br>


In [ ]:
x = [i for i in range(-5, 1, 1)]
y = [i * 0 - 1 for i in range(6)]
x_1 = [i for i in range(0, 6)]
y_1 = [i * 0 + 1 for i in range(0, 6)]

plt.figure(figsize=(8, 5))
plt.plot(x, y, "b")
plt.plot(x_1, y_1, "b")
plt.plot(0, 0, "ro")
plt.plot(0, 1, "bo")
plt.plot(0, -1, "bo")
plt.show()

### Max-Loss

$$b = max(x, y)$$

$$b = x~~if~~x > y~~else~~y$$


$$\frac {\delta b} {\delta x} =  \frac {\delta x} {\delta x}~~if~~x > y~~else~~\frac {\delta y} {\delta x} = 1~~if~~x > y~~else~~0$$

Если $x > y$, то он оказал влияние на $b$. Иначе, его вклада в $b$ НЕ БЫЛО - градиент равен 0

 


### SVM-Loss

Из: 

$L_i=\sum_{j\neq y_i}max(0,s_j-s_{y_i}+1)$

Получаем:

$ \nabla_W	L(W) = {1 \over N}\sum_{i=1}^N \nabla_W L_i(x_i, y_i, W)$

##### $\color{blue}{\text{*Не обязательное задание:}}$

Поставлю дополнительно 20 баллов тому, кто графически оформит результаты этого эксперимента и напишет вывод о зависимости качества обучения от learning_rate и batch_size (возможно, нужно будет добавить варианты их сочетаний и/или увеличить количество эпох до 20).

In [ ]:
import os
from IPython.display import clear_output

file_exists = os.path.exists("/content/cifar-10-batches-py")
if file_exists == False:
    !wget https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
    !tar -xzf cifar-10-python.tar.gz
    !mkdir content
    !mv  cifar-10-batches-py content/cifar-10-batches-py
clear_output()

In [ ]:
import numpy as np
import pickle

def unpickle(file):
    with open(file, "rb") as fo:
        dict = pickle.load(fo, encoding="bytes")
    return dict


x_train = np.zeros((0, 3072))
y_train = np.array([])
for i in range(1, 6):
    raw = unpickle(f"content/cifar-10-batches-py/data_batch_{i}")
    x_train = np.append(x_train, np.array(raw[b"data"]), axis=0)
    y_train = np.append(y_train, np.array(raw[b"labels"]), axis=0)

test = unpickle("content/cifar-10-batches-py/test_batch")
x_test = np.array(test[b"data"])
y_test = np.array(test[b"labels"])

labels_eng = [
    "Airplane",
    "Car",
    "Bird",
    "Cat",
    "Deer",
    "Dog",
    "Frog",
    "Horse",
    "Ship",
    "Truck",
]

print(f"x_train shape: {x_train.shape}, y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape}, y_test shape: {y_test.shape}")

In [ ]:
import random

class LinearClassifier():
    def __init__(self, labels, batch_size, random_state=42):
        self.labels = labels  # classes names
        self.classes_num = len(labels)  # num of classes

        np.random.seed(random_state)  
        self.W = (
            np.random.randn(3073, self.classes_num) * 0.0001
        )  # generate random weights, reshape to add bias 
        self.batch_size = batch_size  # batch_size

    def fit(self, x_train, y_train, learning_rate=1e-8):
        loss = 0.0  # reset loss
        train_len = x_train.shape[0]  # num of examples
        indexes = list(range(train_len))  # indexes train_len
        random.shuffle(indexes)  

        for i in range(0, train_len, self.batch_size):  
            idx = indexes[i : i + self.batch_size]   
            x_batch = x_train[idx]  
            y_batch = y_train[idx]  

            x_batch = np.hstack([x_batch, np.ones((x_batch.shape[0], 1))])  # add bias

            loss_val, grad = self.loss(x_batch, y_batch)  # loss and gradient
            self.W -= learning_rate * grad  # update weigths

            loss += loss_val  # loss sum
        return loss / (train_len)  # mean loss

    def loss(self, x, y):
        current_batch_size = x.shape[0]  # batch_size
        loss = 0.0  
        dW = np.zeros(self.W.shape) 
        for i in range(current_batch_size):  
            scores = x[i].dot(self.W)  # vector of shape 10
            correct_class_score = scores[int(y[i])]  
            above_zero_loss_count = 0  
            for j in range(self.classes_num): 
                if j == y[i]:  # predict class
                    continue
                margin = scores[j] - correct_class_score + 1  # loss
                if margin > 0:  
                    above_zero_loss_count += 1  
                    loss += margin  # 
                    dW[:, j] += x[i]  # 
            dW[:, int(y[i])] -= above_zero_loss_count * x[i]  
        loss /= current_batch_size  
        dW /= current_batch_size  
        return loss, dW

    def forward(self, x):
        x = np.append(x, 1)  # add 1 (bias)
        scores = x.dot(self.W)  
        return np.argmax(scores)  

In [ ]:
def validate(model, x_test, y_test, noprint=False):
    correct = 0  
    for i, img in enumerate(x_test):  
        index = model.forward(img)  
        correct += (
            1 if index == y_test[i] else 0
        )  
        if noprint is False:  
            if i > 0 and i % 1000 == 0:  
                print(
                    "Accuracy {:.3f}".format(correct / i)
                )  
    return correct / len(y_test)  

In [ ]:
print("How learning quality depends of speed:")

for lr in [1e-2, 1e-8]:
    for bs in [256, 2048]:

        print("-" * 50, "\n", "learning_rate =", lr, "\tbatch_size =", bs)
        print()
        lc_model = LinearClassifier(labels_eng, batch_size=bs)

        best_accuracy = 0
        for epoch in range(10):
            loss = lc_model.fit(x_train, y_train, learning_rate=lr)
            accuracy = validate(lc_model, x_test, y_test, noprint=True)
            if best_accuracy < accuracy:
                best_accuracy = accuracy
                best_epoch = epoch
            print(f"Epoch {epoch} \tLoss: {loss}, \tAccuracy:{accuracy}")

        print()
        print(f"Best accuracy is {best_accuracy} in {best_epoch} epoch")

## Выбор шага обучения

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/out/update_weghts_values.png" width="450"> 

<!-- [Визуализация](https://docs.google.com/file/d/0Byvt-AfX75o1ZWxMRkxrUFJ2ZUE/preview)
 -->



Шаг обучения - некоторый коэффициент, как правило, небольшой, который не позволяет нам двигаться слишком быстро. У нас есть точка, в которую мы хотим попасть. Если мы сделаем слишком большой шаг, то мы ее перескочим (график справа), поэтому надо подобрать шаг, который не позволит ее перескочить, но в то же время, чтобы тот же процесс не шел слишком медленно (как на графике слева)

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/out/learning_rate_optimal_value.png" > 

Выбор слишком большого шага обучения может стать причиной нежелательного характера изменения значения функции потерь в процессе обучения. Вместо ожидаемого движения в сторону минимума, функция ошибок может начать периодически изменяться в течение цикла обучения или же вовсе продемонстрировать неограниченный рост.

В качестве иллюстрации этого эффекта, рассмотрим модельный пример. Предположим, мы решаем задачу регрессии и хотим восстановить значение неизвестной скалярной функции. 

Создадим синтетический датасет для функции
$y(x_1, x_2, ... , x_n) = \sum_i x_i^3$

In [ ]:
np.random.seed(101)
x_train = np.random.rand(1000, 5)
noise = np.random.rand(1000, 1) / 1000
y_train = np.expand_dims(np.sum(x_train ** 3, axis=-1), axis=1) + noise

print(f"x_train shape: {x_train.shape}, y_train shape: {y_train.shape}")


Определим простейшую модель линейной регрессии. Используем MSE в качестве функции потерь:

In [ ]:
class LinearRegressor():
    def __init__(self, in_features, out_features, batch_size, random_state=42):
        self.in_features = in_features  # num of inputs
        self.out_features = out_features  # num of outputs

        np.random.seed(random_state)  
        self.W = (
            np.random.randn(self.in_features + 1, self.out_features) * 0.0001
        )  # generate random weights, reshape to add bias 
        self.batch_size = batch_size  # batch_size

    def fit(self, x_train, y_train, learning_rate=1e-8):
        loss = 0.0  # reset loss
        train_len = x_train.shape[0]  # num of examples
        indexes = list(range(train_len))  # indexes train_len
        random.shuffle(indexes)  

        for i in range(0, train_len, self.batch_size):  
            idx = indexes[i : i + self.batch_size]   
            x_batch = x_train[idx]  
            y_batch = y_train[idx]  

            x_batch = np.hstack(
                [x_batch, np.ones((x_batch.shape[0], 1))]
            )  # add bias

            loss_val, grad = self.loss(x_batch, y_batch)  # loss and gradient
            self.W -= learning_rate * grad  # update weigths

            loss += loss_val  # loss sum
        return loss / (train_len)  # mean loss

    def loss(self, x, y):
        '''
        MSE loss
        '''
        current_batch_size = x.shape[0]  # batch_size
        loss = 0.0  
        dW = np.zeros(self.W.shape)
        for i in range(current_batch_size):  
            y_preds_on_batch = x[i].dot(self.W)  # vector of shape out_features
            y_true_on_batch = y[i]
            loss += np.sum((y_preds_on_batch - y_true_on_batch) ** 2)
            # dW_{m, n} = 2 * x_m ((Wx)_n - y_n)
            dW += np.meshgrid(y_preds_on_batch - y_true_on_batch, x[i])[0]
            
        loss /= current_batch_size  
        dW /= current_batch_size  
        return loss, dW

    def forward(self, x):
        x = np.append(x, 1)  # add 1 (bias)
        scores = x.dot(self.W)  
        return np.argmax(scores)  

Обучим нашу модель на синтетических данных и посмотрим, как будет изменяться значение функции потерь в ходе процесса обучения на 20 эпохах при использовании для обучения различных значений скорости обучения.

In [ ]:
from tqdm.notebook import tqdm

def train_and_plot_history(lerning_rates_list, batch_size=64, n_epochs=20):
    fig, ax = plt.subplots(1, 1, figsize=(12, 6))
    for lr_id, lr in tqdm(enumerate(lerning_rates_list)):
        regressor = LinearRegressor(
            in_features=x_train.shape[1], 
            out_features=y_train.shape[1],
            batch_size=42
        )
        loss_hist = np.zeros(n_epochs)
        for epoch in range(n_epochs):
            loss = regressor.fit(x_train, y_train, learning_rate=lr)
            loss_hist[epoch] = loss
        ax.plot(loss_hist, label='lr={:0.1e}'.format(lr))
    
    ax.set_xticks(range(n_epochs))
    ax.set_xlabel('epoch')
    ax.set_ylabel('MSE loss')
    # ax.set_xlim([3, 8])
    plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
    plt.show()

При значениях скорости обучения в диапазоне $5 \times 10^{-4} - 7 \times 10^{-4}$ мы наблюдаем ожидаемый процесс обучения модели:

In [ ]:
train_and_plot_history(
    lerning_rates_list=np.linspace(start=5e-4, stop=7e-4, num=4)
)

Понизив на несколько порядков скорость обучения, мы увидим, что за первые 20 эпох обучения модель существенно не смола улучшить величину MSE:

In [ ]:
train_and_plot_history(
    lerning_rates_list=np.linspace(start=5e-9, stop=7e-9, num=4)
)

Если же мы наоборот увеличим на несколько порядков скорость обучения, то заметим что функция ошибок стала изменяться хаотично и с дальнейшим ростом скорости обучения будет стремиться неограниченно ухудшаться:

In [ ]:
train_and_plot_history(
    lerning_rates_list=np.linspace(start=0.5, stop=0.55, num=4)
)

Пока изменения loss функции достаточно большие, сам по себе градиент тоже большой. За счет этого можно двигаться быстро. Когда он будет уменьшаться, мы должны оказаться рядом с этим минимумом, и шаг, который мы выбрали, не должен мешать этому процессу. 

Бывают ситуации, когда шаг можно менять в самом процессе обучения, когда в начале обучения модели шаг большой, потом, по мере того, как она сходится, чтобы более четко найти минимум, шаг можно изменить вручную. Но изначально его нужно каким-то образом подобрать, и это зависит от данных и от самой модели. Это тоже гиперпараметр, связанный с обучением.

## Выбор размера батча



<img src ="https://edunet.kea.su/repo/EduNet-content/L02/out/define_size_of_batch.png" width="500"> 

Мы с самого начала говорили о выборках, некоторого количества примеров. В дальнейшем, мы будем называть их батчами. Батч (англ. *batch*) - это некоторое подмножество обучающей выборки фиксированного размера.

При этом было не очень понятно, чем они мотивированны. Точнее, мы мотивировали это тем, что у нас много данных, и мы не сможем их обработать все, и это правда. Даже если мы сможем загрузить все данные в память, нам нужно будет загрузить их и использовать при расчете, в том числе градиента. Это ещё более затратно. 

Зачем батчи нужны при рассчете loss? Если мы посчитаем loss по одному изображению, скорее всего он будет очень специфичен, и это движение, которое произойдет, будет направлено в сторону минимума, потому что по этому конкретному изображению мы улучшим показатели, что не отражает обобщения всех данных, поэтому мы используем батчи.

Нас в первую очередь интересует более общее направление, которое будет работать на большинстве наших данных. Поэтому если мы будем оптимизировать модели, исходя из одного элемента данных, путь будет витиеватый, и процесс будет происходить достаточно долго. Если же данные не помещаются в память, то **можно использовать батч того размера, который у нас есть**. Можно загрузить в память и считать по батчам градиент. В этом случае спуск будет более плавным, чем по одному изображению. 

Также при использовании всего датасета тоже есть свои минусы. **Не всегда загрузка всего датасета приводит к увеличению точности**. 

Подробнее: 
* [Методы оптимизации нейронных сетей
](https://habr.com/ru/post/318970/)
* [Обучение нейронной сети](https://alxmamaev.medium.com/deep-learning-на-пальцах-часть-1-341cfe021ef9)


## Регуляризация



__Сложность модели__ (*model complexity*) &mdash; важный гиперпараметр. В частности, для линейной модели, сложность может быть представлена количеством параметров, для полиномиальных моделей &mdash; степенью полинома, для деревьев решений &mdash; глубиной дерева и т.д.

Сложность модели тесно связана с __ошибкой обобщения__ (_generalization error_). Ошибка обобщения отличается от ошибки обучения, измеряемой на тренировочных данных, тем, что позволяет оценить обобщающую способность модели, приобретенную в процессе обучения, давать точные ответы на неизвестных ей объектах. Cлишком простой модели не будет хватать мощности для обобщения сложной закономерности в данных, что приводит к большой ошибке обобщения, с другой стороны слишком сложная модель также приводит к большой ошибке обобщения за счет того, что в силу своей сложности модель начинает пытаться искать закономерности в шуме, добиваясь большей точности на тренировочных данных, теряя при этом часть обобщающей способности.

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/out/model_complexity.png" width="500"> 

Проиллюстрируем описанное явление на примере полиномиальной модели.

In [ ]:
x = np.linspace(0, 2 * np.pi, 10)
y = np.sin(x) + np.random.normal(scale=0.25, size=len(x))
plt.scatter(x, y, s=50, facecolors='none', edgecolors='b', label='noisy data')

x_true = np.linspace(0, 2 * np.pi, 200)
y_true = np.sin(x_true)
plt.plot(x_true, y_true, c='lime', label='ground truth')
plt.legend()
plt.show()

Попробуем аппроксимировать имеющуюся зависимость с помощью полиномиальной модели, используя шумные данные в качестве тренировочных данных:

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

x_train = x.reshape(-1, 1)

fig = plt.figure(figsize=(12, 6))

for i, degree in enumerate([0, 1, 3, 9]):

    model = make_pipeline(PolynomialFeatures(degree), LinearRegression())

    model.fit(x_train, y)
    y_plot = model.predict(x_true.reshape(-1, 1))

    fig.add_subplot(2, 2, i + 1)
    plt.plot(x_true, y_plot, c='red', label=f'M={degree}')
    plt.scatter(x, y, s=50, facecolors='none', edgecolors='b')
    plt.plot(x_true, y_true, c='lime')
    plt.legend()
plt.show()

Видно, что модель может переобучаться, подстраиваясь под тренировочную выборку. В полиноме степень, и как следствие количество весов &mdash; это гиперпараметр, который можно подбирать на кросс-валидации, однако, когда мы, таким образом, подбираем сложность модели мы налагаем довольно грубое ограничение на обобщающую способность модели в целом. Вместо этого более разумным было бы оставить модель сложной, но использовать некий ограничитель (__регуляризатор__), который будет заставлять модель отдавать предпочтение выбору более простого обобщения. 

In [ ]:
from sklearn.linear_model import Ridge

model = make_pipeline(PolynomialFeatures(9), LinearRegression())
model_ridge = make_pipeline(PolynomialFeatures(9), Ridge(alpha=0.1))

model.fit(x_train, y)
y_plot = model.predict(x_true.reshape(-1, 1))

model_ridge.fit(x_train, y)
y_plot_ridge = model_ridge.predict(x_true.reshape(-1, 1))

plt.plot(x_true, y_plot, c='red', label=f'M={degree}')
plt.plot(x_true, y_plot_ridge, c='black', label=f'M={degree}, alpha=0.1')
plt.scatter(x, y, s=50, facecolors='none', edgecolors='b')
plt.plot(x_true, y_true, c='lime', label='ground truth')
plt.legend()
plt.show()

poly_coef = model[1].coef_

eq = f'y = {round(poly_coef[0], 2)}+{round(poly_coef[1], 2)}*x'
for i in range(2, 10):
    eq += f'+{round(poly_coef[i], 2)}*x^{i}'
    
print('Without regularization: ', eq)

poly_coef = model_ridge[1].coef_

eq = f'y = {round(poly_coef[0], 2)}+{round(poly_coef[1], 2)}*x'
for i in range(2, 10):
    eq += f'+{round(poly_coef[i], 2)}*x^{i}'

print('With regularization: ', eq)

Видно, что одним из "симптомов" переобучения являются аномально большие веса. Модель Ridge Regression, показанная в примере выше, использует L2 регуляризуцию для борьбы с этим явлением.




<img src ="https://edunet.kea.su/repo/EduNet-content/L02/out/l2_regularization.jpg" width="700"> 


L2 Regularization = weights decay

Идея состоит в том, что мы можем наложить некоторое требование на сами веса. Дело в том, что можно получить один и тот же выход модели при разных весах (выход модели соответствует умножению весов на $x$), при разных $w$ выход может быть идентичен.

Эти параметры задают некоторую аппроксимацию нашей целевой функции, аппроксимировать функцию можно двумя способами:
1. Использовать все имеющиеся данные и провести ее строго через все точки, которые нам известны; 
2. Использовать более простую функцию: (в данном случае линейную), которая не попадет точно во все данные, но зато будет соответствовать некоторым общим закономерностям, которые у них есть. 

Характерной чертой переобучения является второй сценарий, и сопровождается он, как правило, большими весами. Введение L2-регуляризации приводит к тому, что большие веса штрафуются и предпочтение отдается решениям, использующим малые значения весов. 

Модель может попробовать схитрить и по-другому - использовать все веса, все признаки, даже незначимые, но с маленькими коэффициентами. С этим L2-loss поможет хуже, так как он не сильнее штрафует мелкие веса. Результатов его применения - малые значения весов, которые использует модель

В этом случае на помощь приходит L1-loss, который штрафует вес за сам факт отличия его от нуля. Но и штрафует он все веса одинаково. Результат его применения - малое число весов, которые использует модель в принципе. 



<img src="https://edunet.kea.su/repo/EduNet-content/L02/out/l1_and_l2_regularization.gif" alt="alttext" width="550"/>


Это лоссы можно комбинировать - получится Elastic Net




$\lambda=$ regularization strength (hyperparameter)

$L(W)=\underbrace{\frac1N\sum_{i=1}^NL_i(f(x_i,W),y_i)}_{\textbf{Data loss} }+\underbrace {\lambda R(W)}_{\textbf{Regularization}}$

Берем сумму всех весов по всей матрице $w$, и добавляем ее к loss. Соответственно, чем больше будет эта сумма, тем больше будет суммарный loss. 


В дальнейшем проблема с переобучением будет вставать довольно часто. Методов регуляризации модели существует достаточно много. Этот — один из базовых, который будет использоваться практически во всех оптимизаторах, с которыми познакомимся позже.

# Функция потерь Кросс-энтропия 

[Отличное видео от Statquest про энтропию](https://www.youtube.com/watch?v=YtebGVx-Fxw)



На Cross-entropy Loss построены практически все модели, про которые будет идти речь.

Это оценка результата и обновление весов с использованием градиента. У нас один слой, поэтому мы достаточно легко посчитали от него градиент. Если слоев будет много, сделать это будет сложней.

Рассмотрим ещё одну loss-функцию. 

SVM loss хороша тем, что интуитивно понятно, как ее посчитать. Но проблема состоит в том, что модель выдает некоторые числа, которые сами по себе невозможно интерпретировать. Сами по себе выходы модели мало что означают. Было бы неплохо, если бы мы сразу, глядя на выход модели для кошки, могли бы их как-то интерпретировать, не просматривая остальные веса. Хорошо бы, чтобы модель выдавала не какую-то абстрактную уверенность, а **вероятность** того, что по ее мнению, на картинке изображена кошка.


## Переход к вероятностям

**Softmax**

[Видео от StatQuest, которое объясняет Softmax](https://www.youtube.com/watch?v=KpKog-L9veg)

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/out/scores_to_probability.png" width="750">

Перейти к вероятностям мы сможем, проведя с весами некоторые не очень сложные математические преобразования. 

На слайде выше показано, почему выходы модели часто называют [logit’ами](https://en.wikipedia.org/wiki/Logit). Если предположить, что у нас есть некая вероятность, от которой мы берем такую функцию (logit), то она может принимать значения от нуля до бесконечности. Мы можем считать, что выходы модели - это logit’ы. 




Например, мы могли бы просто взять индекс массива, в котором значение (logit) максимально. Предположим, что наша сеть выдала следующие значения:

In [ ]:
logits = [
    5.1, # cat
    3.2, # car
    -1.7, # frog
]

Тогда, чтобы узнать какой класс наша сеть предсказала, мы могли бы просто взять `argmax` от наших `logits`

In [ ]:
import numpy as np

print('Predicted class = %i (Cat)' % (np.argmax(logits)))

Но от argmax нельзя посчитать градиент, так как производная от константы равна 0. Соответственно, если бы мы вставили производную от argmax в градиентный спуск, мы бы получили везде нули, и соответственно, наша сеть бы вообще ничему не научилась

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(np.arange(3), [1, 0, 0], color='red', s=50)
plt.show()

А мы бы хотели получить не logit’ы, а настоящую вероятность на выходе модели. Да еще и таким образом, что бы от наших вероятностей можно было бы посчитать градиент. Для этого мы можем применить к нашим логитам функцию **Softmax**

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/out/linear_classifier_softmax.png" width="1300">

Мы можем провести над логитами операцию экспоненцирования, то есть число Эйлера (2.71828) **возвести в степень**, соответствующую этому выходу. В результате, мы получим вектор, гарантировано неотрицательных чисел (потому что мы ввели неположительное число в степень, пускай даже отрицательную, то есть выходы могут быть маленькие, но всегда положительные). 



Дальше, чтобы можно было интерпретировать эти числа как вероятности, их сумма должна быть равна единице. Мы должны их нормализовать, то есть **поделить на сумму**. Это преобразование называется **Softmax функцией**.

**Получаются вероятности**, то есть числа, которые можно интерпретировать таким образом.

$\text{Softmax}_\text{кошка} = \frac{e^{5.1}}{e^{5.1} + e^{3.2} + e^{-1.7}}$

In [ ]:
def softmax(logits):
    return np.exp(logits) / np.sum(np.exp(logits))

print(softmax(logits))
print('Sum = %.2f' % np.sum(softmax(logits)))

Можно обратить внимание, что Softmax, никоим образом не поменял порядок значений. Самому большому logit'у соответствует самая большая вероятность, а самому маленькому, соответственно самая маленькая

Посмотрим на графиках. Возьмем массив случайных логитов и применим к ним softmax

In [ ]:
rand_logits = np.linspace(-1, 1, 50)
fig,ax = plt.subplots(ncols=2)

ax[0].plot(np.arange(50), rand_logits)
ax[0].set_title('Logits')
ax[1].plot(np.arange(50), softmax(rand_logits))
ax[1].set_title('Softmax')
plt.show()

**Cross-entropy / log loss**

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/out/cross_entropy_plot_loss_with_probability.png" width="800">


Теперь, с использованием такого вектора можно определить другую loss функцию. Если не вникать в детали, можно **взять от нее логарифм**. Тогда loss от такого выхода будет выглядеть вот таким образом:

$ L_i =  -log(\frac{e^{s_{y_i}}}{\sum_j e^{s_j}})$

как нормализованный выход от верного класса на сумму остальных, к которому добавили минус. 

Получится график loss (слева). Его плюс заключается в том, что у него нет участка с плато, практически по всей длине функция получается гладкой, с хорошими мощными производными. Когда loss большой, производная тоже большая, и за счет этого можно быстро обучать модель. Для кусочно-линейной функции потерь она же равна константе.

До тех пор, пока модель не будет работать с точностью 100%, то есть пока все остальные выходы не станут нулевыми, мы можем продолжить обучение, даже если у нас нет явной регуляризации.

Осталось выяснить, почему такая простая на вид функция **называется так сложно** - Кросс-энтропия. 



## Определение Кросс-энтропии




Начнем с простого примера, пусть у нас есть 10 точек со следующими значениями признака x: 

`x = [-2.2, -1.4, -0.8, 0.2, 0.4, 0.8, 1.2, 2.2, 2.9, 4.6]`

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/out/cross_entropy_ten_dots.png">

Пусть наши точки принадлежат двум классам: зеленый и красный:

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/out/cross_entropy_two_classes.png">

Перед нами простая задача классификации: по признаку x предсказать класс наших точек. Мы можем переформулировать задачу как нахождение вероятности того, что точка зеленая или красная. В идеальной ситуации для зеленой точки вероятность того, что она зеленая равна 1, в то же время вероятность того, что красная точка &mdash; зеленая должна быть равна 0. 

Вероятности, которые выдает, обучаемая нами модель, зачастую далеки от идеальной ситуации. В нашем примере сравнить насколько сильно предсказанная вероятность класса отличается от вероятности, выдаваемой "идеальной моделью" можно за счет __бинарной кросс-энтропии__ частного случая кросс-энропии.

$$H_p(q)=-\frac{1}{N}\sum^N_{i=1}y_i\cdot log(p(y_i))+(1-y_i)\cdot log(1-p(y_i))$$

где $y$ &mdash; метка класса (1 для зеленого, 0 для красного), которую можно также интерпретировать как вероятность, предсказанную "идеальной моделью", $p(y)$ &mdash; вероятность того, что точка зеленая, предсказываемая оцениваемой моделью.

Какое же отношение энтропия имеет к этой формуле? Давайте углубимся в детали.

Поскольку $y$ представляет метку классов точек, то его распределение $q(y)$ выглядит следующим образом:

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/out/cross_entropy_distribution.png">

__Энтропия__ &mdash; мера неуверенности, связанная с распределением $q(y)$. Какова была бы мера неуверенности распределения $q(y)$ если бы все точки были зелеными? Так как у нас бы не было сомнений насчет цвета точки (он всегда зеленый), значение энтропии было бы 0. Теперь представим другую ситуацию, пусть у нас поровну точек зеленого и красного цвета. Для нас это наихудшая ситуация, поскольку попытка определить цвет точки, по сути, представляет случайное угадывание. В этом случае энтропия вычисляется по формуле Хартли:

$$H(q)=log(2)$$

Мы рассмотрели два крайних случая, но как быть с промежуточными ситуациями? Для этих случаев мы можем использовать формулу Шеннона:

$$H(q)=-\sum^C_{c=1}q(y_c)\cdot log(q(y_c))$$

где C &mdash; количество классов. Нетрудно заметить, что формула Хартли является частным случаем формулы Шеннона.

Таким образом, зная истинное распределение случайной величины, мы можем рассчитать его энтропию. А что будет если мы попытаемся аппроксимировать истинное распределение $q(y)$ некоторым другим распределением $p(y)$? Допустим, что наши цветные точки подчиняются этому распределению $p(y)$, также мы знаем, что исходят они из неизвестного нам истинного распределения $q(y)$, если мы посчитаем следующую энтропию, это и будет __кросс-энтропия__:

$$H_p(q)=-\sum^C_{c=1}q(y_c)\cdot log(p(y_c))$$

Если окажется, что распределения $p(y)$ и $q(y)$ совпадают, в этом случае энтропия $H(q)$ и кросс-энтропия $H_p(q)$ также будут совпадать. Однако в реальности такое случается редко и кросс-энтропия бывает больше энтропии истинного распределения

$$H_p(q)-H(q)\geq0$$

Разница между кросс-энтропией и энтропией называется __дивергенцией Кульбака-Лейблера__, которая является мерой различия между двумя распределениями:

$$D_{KL}(q||p)=H_p(q)-H(q)=\sum^C_{c=1}q(y_c)\cdot [log(q(y_c))-log(p(y_c))]$$

Данная формула для дивергенции Кульбака-Лейблера по своей сути является записью выражения для __математического ожидания величины__ $log(q(y_c))-log(p(y_c)) = \log(\frac{q(y_c)}{p(y_c)})$ (__логарифма отношения вероятностей $p(y)$ и $q(y)$__) , __взятого по "истинному" распределению $q(y)$__. Это значит, что чем ближе распределение вероятности $p(y)$ к распределению $q(y)$, тем меньше будет значение дивергенции Кульбака-Лейблера и, следовательно, меньше значение кросс-энтропии. 

При построении моделей классификации можно определить вероятностные распределения $p(x)$ и $q(x)$. Действительно, для обучающей выборки (для которой мы читаем известным номера правильных классов для всех объектов) мы можем приписать вероятность принадлежности некоторого объекта к его истинному классу равной $1$, а вероятность принадлежности ко всем другим классам положить равной $0$ . Например, в случае бинарной классификации $q(y)$ представляет собой просто индикатор принадлежности объекта к одному из классов:
$$q(y) =  \begin{cases} 1 \, \text{if $y \in C$}  \\ 0 \, \text{else} \end{cases}$$

Если выходы модели принимают значения в вещественных числах, то их также можно интерпретировать как вероятности, переменив описанное выше SoftMax отображение. Если $\vec z$ -- вектор выходов модели, то величины:
$$ \sigma(\vec z)_i = \frac{e^{z_i}}{\sum_j e^{z_j}} $$
дадут искомое распределение $p(y)$.


Из определения выше можно легко получить выражение для кросс-энтропии пары вероятностных распрелелений, записанных через соответствующие энтропию и дивергенцию Кульбака-Лейблера:
$$H_p(q) = D_{KL}(q||p) + H(q)$$.

Важно заметить, что энтропия априорного распределения равна нулю $H(q)$ (так как равны нулю все слагаемые). То есть численное значение близости оприорного $q$ и оценочного $p$ распределений вероятности по дивергенции Кульбака-Лейбера даётся просто величиной кросс-энтропии $H_p(q)$. Мы хотим построить модель классификации, которая бы давала верную оценку принадлежности объектов классам — т. е. порождала $p(y)$ близкое к $q(y)$. Для этих целей мы стремимся __минимизировать кросс-энтропию__ при обучении модели.


Таким образом, можно коротко сформулировать "рецепт" построения модели классификации с использованием кросс-энтропии в качестве функции потерь:

* Построить априорное распределение вероятности принадлежности объектов классам для размеченных данных $Q$
* Преобразовать "сырые" выходы модели в нормализованные и нормированные величины, которые можно будет интерпретировать в качестве оценок вероятностей принадлежностей исследуемых объектов к классам — построить оценочное распределение $P$
* Сравнить априорные и оценочные вероятности принадлежности классам, вычислив между ними кросс-энтропию $H(P|Q)$

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/out/logits_to_scores_to_probabilitys.png" width="900">



### Дополнение: кросс-энтропия с точки зрения теории информации

Рассмотрим следующий пример:

Пусть у нас есть некоторые данные, которые мы передаем по каналу связи. Например, у нас есть метеостанция, которая сообщает прогноз погоды. Допустим, она может передавать 8 вариантов прогноза. Мы в каждый момент времени получаем от нее сообщение. Для передачи всех 8 возможных вариантов прогноза погоды наше сообщение должно содержать не менее 3 бит информации. Можем ли мы эффективнее распорядиться нашим каналом связи и в среднем тратить меньшее количество бит информации на передачу сообщений? [*Оказывается, это возможно*](https://en.wikipedia.org/wiki/Shannon%E2%80%93Fano_coding). Но необходимы некоторые дополнительные предположения.

Допустим, мы знаем некую вероятность, с которой в нашем регионе может наступить хорошая либо плохая погода. Пусть у имеет место вероятностное распределение, по которому у нас, допустим, солнечное место, где почти не бывает бури. Например, как на нижнем правом изображении на следующей иллюстрации:

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/out/efficient_way_to_send_signals.png" >


Если мы знаем об этом, мы можем сэкономить на передаче информации. Мы можем закодировать наиболее вероятные сообщения двумя битами, причем придумать такие коды, чтобы они почти не пересекались. Они будут начинаться с нуля и передавать два бита вместо трех. Таким образом, можно сэкономить на передачи информации. То есть идея такая: **если мы знаем, что события маловероятны, мы можем кодировать их более длинной цепочкой, а более вероятные события - более короткими цепочками**. В этом случае в среднем количество информации, которое можно передать, сократится.

Обратимся вновь к формуле Шеннона:

$$H(q)=-\sum_{x \in X}q(x)\cdot \log(q(x)),$$

в которую подставим изображенное на рисунке вероятностное распределение:

$$H(Q)= - (2 \cdot 0.35 \cdot \log_2(0.35) + 2 \log_2 0.10 \cdot \log_2(0.10) + 2 \cdot 0.04 \cdot \log_2(0.04) + 2 \cdot 0.01 \cdot \log_2(0.01)) = 2.23.$$

Мы получили число несколько меньшее $3$. [Можно показать](https://www.youtube.com/watch?v=YtebGVx-Fxw), что эта величина $2.23$ даёт оценку среднего количества информации, "содержащегося" в одном сообщении. На практике реализовать это, используя биты, возможно, не выйдет, но это свойство данного вероятностного распределения можно посчитать. Оно несёт в себе некоторое количество полезной информации, соответствующее этой энтропии.


Предложенная кодовая система явно опирается на априорную информацию о распределении частот событий. Что будет, если частота событий изменится (или же мы просто будем использовать определенную выше систему кодирования для формирования сообщений о погоде в другом месте, где климат может отличаться; см. нижний левый фрагмент рисунка)? При смене местоположения метеостанции с сохранением схемы кодирования событий, у нас осталось неизменным априорное распределение $q$ частот погодных событий из старого места, но появилось некоторое новое "оценочное" распределение $p$ -- распределение частот погодных классов в новом месте. Вычислим кросс-энтропию между этими распределениями:

$$H(P|Q)= - (2 \cdot 0.35 \cdot \log_2(0.01) + 2 \log_2 0.10 \cdot \log_2(0.04) + 2 \cdot 0.04 \cdot \log_2(0.10) + 2 \cdot 0.01 \cdot \log_2(0.35)) = 5.84.$$

[В теории информации](https://people.math.harvard.edu/~ctm/home/text/others/shannon/entropy/entropy.pdf) кросс-энтропия между двумя распределениями вероятностей $p$ и $q$  по одному и тому же базовому набору событий измеряет среднее число битов, необходимых для идентификации события, взятого из набора, если схема кодирования, используемая для набора, оптимизирована для оценочного распределения вероятностей $q$, а не для истинного распределения $p$. Вычтя теперь из полученного значения $H(P|Q)$ величину информационной энтропии $H(Q)$ получим описанную выше дивергенцию Кульбака-Лейблера:

$$D_{KL}(P|Q) = H(P|Q) - H(Q) \approx 3.64$$,
которая оказалась существенно отлична от нуля. Большое значение $D_{KL}(P|Q)$ позволяет нам убедиться в том, что вероятностные распределения $P$ и $Q$ существенно отличаются и нам не стоит переносить схему кодирования старой метеостанции на новую (подумайте, почему?). 


## Градиент функции потерь Кросс-энтропии




[Cross Entropy Loss](https://wandb.ai/wandb_fc/russian/reports/---VmlldzoxNDI4NjAw#:~:text=%D0%A4%D1%83%D0%BD%D0%BA%D1%86%D0%B8%D1%8F%20%D0%BF%D0%BE%D1%82%D0%B5%D1%80%D1%8C%20%D0%BF%D0%B5%D1%80%D0%B5%D0%BA%D1%80%D0%B5%D1%81%D1%82%D0%BD%D0%BE%D0%B9%20%D1%8D%D0%BD%D1%82%D1%80%D0%BE%D0%BF%D0%B8%D0%B8%20%E2%80%93%20%D1%8D%D1%82%D0%BE,%2C%20%D0%B3%D0%B4%D0%B5%200%20%E2%80%93%20%D0%B8%D0%B4%D0%B5%D0%B0%D0%BB%D1%8C%D0%BD%D0%B0%D1%8F%20%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D1%8C.)

$$ -L = \sum_i y_i \log p_i = \sum_i y_i \log(\dfrac {e^{s_{y_i}}} {\sum_j e^{s_{y_j}}})$$

$$s_{y_i} = w_i x$$

$$ \dfrac {\delta L} {\delta w_i} = \dfrac {\delta L} {\delta s_{y_i}} \dfrac {\delta s_{y_i}} {\delta w_i} $$

$$\dfrac {\delta s_{y_i}} {\delta w_i} = x$$

Только один $y_k = 1$


$$ -L = y_k \log p_i = \log(\dfrac {e^{s_{y_i}}} {\sum_j e^{s_{y_j}}})$$

i = k

$$ -L = \log(\dfrac {e^{s_{y_i}}} {\sum_j e^{s_{y_j}}}) = \log e^{s_{y_i}} - \log  \sum_j e^{s_{y_j}}  = s_{y_i} - \log  \sum_j e^{s_{y_j}}$$

$$\dfrac {\delta -L} {\delta s_{y_i}} = 1 - \dfrac 1 {\sum_j e^{s_{y_j}}} \cdot \dfrac {\delta {\sum_j e^{s_{y_j}}}} {\delta s_{y_i}} = 1 - \dfrac 1 {\sum_j e^{s_{y_i}}} \cdot \dfrac {\delta e^{s_{y_i}}} {\delta s_{y_i}} = 1 - \dfrac {e^{s_{y_i}}} {\sum_j e^{s_{y_j}}} = 1 - p_i$$

$$\dfrac {\delta L} {\delta s_{y_j}} = p_i - 1 $$

$$ \dfrac {\delta L_i} {\delta w_i}  = \dfrac {\delta L} {\delta s_{y_i}} \dfrac {\delta s_{y_i}} {\delta w_i} = (p_i - 1) x $$

i != k

$$ -L = \log(\dfrac {e^{s_{y_i}}} {\sum_j e^{s_{y_j}}}) = \log e^{s_{y_k}} - \log  \sum_j e^{s_{y_j}}  = s_{y_k} - \log  \sum_j e^{s_{y_j}}$$

$$\dfrac {\delta -L} {\delta s_{y_i}} = - \dfrac 1 {\sum_j e^{s_{y_j}}} \cdot \dfrac {\delta {\sum_j e^{s_{y_j}}}} {\delta s_{y_i}} =  \dfrac 1 {\sum_j e^{s_{y_i}}} \cdot \dfrac {\delta e^{s_{y_i}}} {\delta s_{y_i}} = \dfrac {e^{s_{y_i}}} {\sum_j e^{s_{y_j}}} = - p_i$$

$$\dfrac {\delta L} {\delta s_{y_j}} = p_i $$
$$ \dfrac {\delta L_i} {\delta w_i}  = \dfrac {\delta L} {\delta s_{y_i}} \dfrac {\delta s_{y_i}} {\delta w_i} = p_i x $$



В коде это будет выглядеть вот так:

In [ ]:
# Input batch of 2 vector with 4 elements
x = np.array([ 
              [1, 2, 3, 4], 
              [1, -2, 0, 0]
            ])
# Weights
W = np.random.randn(3, 4) # 3 class

# model output
logits = x.dot(W.T)
print("Scores(Logits) \n",logits,"\n")

# Probabilities
probs = softmax(logits) #defined before
print("Probs \n",probs,"\n")

# Ground true classes
y = [0,1] 

# Derivative
probs[np.arange(1),y] = -1 # substract one from true class prob
dW = x.T.dot(probs) # dot product with input

print("Grads dL/dW \n",dW) # have same shape as W


## Практическое вычисление SoftMax

Когда мы возводим число в степень, может получиться очень большое число и произойти переполнение памяти. Неизвестно, какие будут у модели выходы. 

In [ ]:
from warnings import simplefilter
simplefilter("ignore", category=RuntimeWarning)

f = np.array([123, 456, 789])
p = np.exp(f) / np.sum(np.exp(f))
print(p)

Поэтому мы можем вычесть из каждого $s_i$ положительную константу, чтобы уменьшить значения экспонент. В качестве константы можно выбрать максимальный элемент этого вектора, тогда у нас гарантированно не будет очень больших чисел, и такой способ будет работать более стабильно.

$$M = \max_j s_{y_{j}}$$
$$s^{new}_{y_{i}}  = s_{y_{i}} - M $$

$$ \dfrac {e^{s^{new}_{y_{i}}}} {\sum_j e^{s^{new}_{y_{j}}}}  = \dfrac {e^{s_{y_{i}} - M }} {\sum_j e^{s_{y_{j}} - M }} = \dfrac {e^{s_{y_{i}}}e ^ {-M}} {\sum_j e^{s_{y_{j}}} e ^ {-M}} = \dfrac {e ^ {-M} e^{s_{y_{i}}}} {e ^ {-M} \sum_j e^{s_{y_{j}}} } = \dfrac { e^{s_{y_{i}}}} { \sum_j e^{s_{y_{j}}} }$$

In [ ]:
f = np.array([123, 456, 789])
f -= f.max()
p = np.exp(f) / np.sum(np.exp(f))
print(f, p)